In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/FrancescoZanella/RecSysChallenge2024.git

Cloning into 'RecSysChallenge2024'...
remote: Enumerating objects: 3456, done.
remote: Counting objects: 100% (1066/1066), done.
remote: Compressing objects: 100% (393/393), done.
remote: Total 3456 (delta 714), reused 1007 (delta 670), pack-reused 2390
Receiving objects: 100% (3456/3456), 43.22 MiB | 35.87 MiB/s, done.
Resolving deltas: 100% (2209/2209), done.


In [2]:
%cd /kaggle/working/RecSysChallenge2024/src

/kaggle/working/RecSysChallenge2024/src


In [3]:
! pip install --upgrade polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.15
    Uninstalling polars-0.20.15:
      Successfully uninstalled polars-0.20.15


In [4]:
! pip install simsimd scikit-learn==1.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.2/382.2 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import polars as pl
import scipy.stats as stats
import scipy.sparse as sps
import gc
import tqdm

from ebrec.utils._descriptive_analysis import (
    min_max_impression_time_behaviors, 
    min_max_impression_time_history
)
from ebrec.utils._polars import slice_join_dataframes
from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    truncate_history,
)
from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_USER_COL
)
from ebrec.evaluation.metrics_protocols import *
from polimi.utils._catboost import (
    _preprocessing_history_trendiness_scores, 
    add_history_trendiness_scores_feature, 
    _preprocessing_mean_delay_features, 
    add_mean_delays_features, 
    _preprocessing_window_features, 
    add_window_features, 
    add_trendiness_feature_categories, 
    _preprocessing_article_endorsement_feature, 
    get_unique_categories,
    add_trendiness_feature,
    _preprocessing_article_endorsement_feature_by_article_and_user,
)

from rgf.sklearn import FastRGFClassifier
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, TargetEncoder, MinMaxScaler
from sklearn.impute import KNNImputer

2024-05-19 16:53:01.777959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 16:53:01.778058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 16:53:01.912429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Memory Utils

In [6]:
from os import system, getpid, walk
from psutil import Process
from colorama import Fore, Style, init
from IPython.display import display, HTML

def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL)
    
def GetMemUsage():   
    pid = getpid()
    py = Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return f"RAM memory GB usage = {memory_use :.4}"

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 0.8697


In [7]:
def reduce_polars_df_memory_size(df, set_categorical=True):

    start_mem = df.estimated_size('mb')
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type in [pl.Int16, pl.Int32, pl.Int64]:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df = df.with_columns(pl.col(col).cast(pl.Int8))
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df = df.with_columns(pl.col(col).cast(pl.Int16))
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df = df.with_columns(pl.col(col).cast(pl.Int32))
        elif col_type in [pl.UInt16, pl.UInt32, pl.UInt64]:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt8))
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt16))
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt32))
        elif col_type == pl.Float64:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df = df.with_columns(pl.col(col).cast(pl.Float32))

    gc.collect()
    end_mem = df.estimated_size('mb')
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# Data import

In [8]:
articles = pl.read_parquet('/kaggle/input/ebnerd_small/articles.parquet')

behaviors_train = pl.read_parquet('/kaggle/input/ebnerd_small/train/behaviors.parquet')
history_train = pl.read_parquet('/kaggle/input/ebnerd_small/train/history.parquet')

behaviors_val = pl.read_parquet('/kaggle/input/ebnerd_small/validation/behaviors.parquet')
history_val = pl.read_parquet('/kaggle/input/ebnerd_small/validation/history.parquet')

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 1.129


# Articles Preprocessing

In [9]:
articles = articles.with_columns(
    pl.col('image_ids').list.len().alias('num_images'),
    pl.col('title').str.split(by=' ').list.len().alias('title_len'),
    pl.col('subtitle').str.split(by=' ').list.len().alias('subtitle_len'),
    pl.col('body').str.split(by=' ').list.len().alias('body_len'),
    # very important for tf-idf, otherwise multiple tokens for topics with spaces are built
    pl.col('topics').list.eval(pl.element().str.split(by=' ').list.join('_')),
    pl.col('topics').list.len().alias('num_topics'),
    (pl.col('total_pageviews') / pl.col('total_inviews')).alias('total_pageviews/inviews')
)
articles = reduce_polars_df_memory_size(articles)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
articles.head(2)

Memory usage of dataframe is 58.23 MB
Memory usage after optimization is: 58.00 MB
Decreased by 0.4%

RAM memory GB usage = 1.137


article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews/inviews
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str,u32,u8,u8,u16,u32,f32
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig_kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative""",1,5,22,200,2,null
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film_og_tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive""",1,5,19,116,3,null


In [10]:
unique_entities = articles.select('entity_groups').explode('entity_groups')['entity_groups'].unique().to_list()
unique_entities = [e for e in unique_entities if e is not None]
unique_entities

['PROD', 'ORG', 'LOC', 'MISC', 'EVENT', 'PER']

In [11]:
# in case of two category_str for the same category, one of the two is picked
unique_categories_df = articles.select(['category', 'category_str']).unique('category').drop_nulls('category')
unique_categories = {
    row['category']: row['category_str'] for row in unique_categories_df.iter_rows(named=True)
}
unique_categories

{414: 'underholdning',
 539: 'ferie',
 2504: 'bibliotek',
 2975: 'penge',
 572: 'side9',
 140: 'krimi',
 457: 'forbrug',
 2889: 'auto',
 2731: 'dagsorden',
 498: 'musik',
 561: 'haandvaerkeren',
 2737: 'podcast',
 2341: 'plus',
 22: 'horoskoper',
 1505: 'vin',
 529: 'biler',
 2: 'incoming',
 731: 'services',
 118: 'nyheder',
 142: 'sport',
 806: 'om_ekstra_bladet',
 2077: 'opinionen',
 512: 'nationen',
 68: 'video',
 565: 'sex_og_samliv'}

Mapping sentiment labels to -1, 0, 1 to save memory in the history dataframe.
Taking only article_default type and mapping it to 0/1 since after only this is needed (and it saves memory) for the history at the moment

In [12]:
articles = articles.with_columns(
    pl.when(pl.col('sentiment_label') == 'Negative').then(-1) \
        .otherwise(
            pl.when(pl.col('sentiment_label') == 'Positive').then(1).otherwise(0)
        ).cast(pl.Int8).alias('sentiment_label_int'),
    (pl.col('article_type') == 'article_default').cast(pl.UInt8).alias('is_article_default')
)

## Topics TF-IDF

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
articles = articles.with_columns(
    pl.Series(
        vectorizer.fit_transform(
            articles.with_columns(pl.col('topics').list.join(separator=' '))['topics'].to_list()
        ).toarray().astype(np.float32)
    ).alias('topics_idf')
)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
articles.head(2)


RAM memory GB usage = 1.149


article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews/inviews,sentiment_label_int,is_article_default,topics_idf
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str,u32,u8,u8,u16,u32,f32,i8,u8,list[f32]
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig_kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative""",1,5,22,200,2,null,-1,1,"[0.0, 0.0, … 0.0]"
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film_og_tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive""",1,5,19,116,3,null,1,1,"[0.0, 0.0, … 0.536059]"


In [14]:
# without split + join with _ of the single topics this returns 94
len(vectorizer.get_feature_names_out())

79

## Topic Model

In [15]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

stopwords = nltk.corpus.stopwords.words('danish')
title_vectorizer = CountVectorizer(stop_words=stopwords)

titles_bow = title_vectorizer.fit_transform(articles['title'].to_list())

In [16]:
%%time

from sklearn.decomposition import LatentDirichletAllocation

# parameters tuned on perplexity
N_COMPONENTS = 5
lda_model = LatentDirichletAllocation(
    n_components=N_COMPONENTS,
    doc_topic_prior=0.99,
    topic_word_prior=0.75,
    learning_method='online'
)
articles = articles.with_columns(
    pl.Series(lda_model.fit_transform(titles_bow).astype(np.float32)) \
        .list.to_array(5).alias('topic_model_embeddings')
)
articles.head(2)

CPU times: user 31.6 s, sys: 3.36 ms, total: 31.6 s
Wall time: 31.6 s


article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews/inviews,sentiment_label_int,is_article_default,topics_idf,topic_model_embeddings
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str,u32,u8,u8,u16,u32,f32,i8,u8,list[f32],"array[f32, 5]"
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig_kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative""",1,5,22,200,2,null,-1,1,"[0.0, 0.0, … 0.0]","[0.287593, 0.143573, … 0.281279]"
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film_og_tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive""",1,5,19,116,3,null,1,1,"[0.0, 0.0, … 0.536059]","[0.532566, 0.137354, … 0.112907]"


In [17]:
topic_model_columns = [f'topic_model_embedding_{i}' for i in range(N_COMPONENTS)]

articles = articles.with_columns(
    pl.col('topic_model_embeddings').arr.to_struct(fields=lambda i: f'topic_model_embedding_{i}')
).with_columns([
    pl.col('topic_model_embeddings').struct.field(f'topic_model_embedding_{i}') for i in range(N_COMPONENTS)
]).drop('topic_model_embeddings')

articles.head(2)

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews/inviews,sentiment_label_int,is_article_default,topics_idf,topic_model_embedding_0,topic_model_embedding_1,topic_model_embedding_2,topic_model_embedding_3,topic_model_embedding_4
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str,u32,u8,u8,u16,u32,f32,i8,u8,list[f32],f32,f32,f32,f32,f32
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig_kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative""",1,5,22,200,2,null,-1,1,"[0.0, 0.0, … 0.0]",0.287593,0.143573,0.143915,0.14364,0.281279
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film_og_tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive""",1,5,19,116,3,null,1,1,"[0.0, 0.0, … 0.536059]",0.532566,0.137354,0.108729,0.108444,0.112907


# History preprocessing

In [18]:
def add_topics_tf_idf_columns(df, topics_col, vectorizer):
    return df.with_columns(
        pl.Series(
            vectorizer.transform(
                df.with_columns(pl.col(topics_col).list.join(separator=' '))[topics_col].to_list()
            ).toarray().astype(np.float32)
        ).alias(f'{topics_col}_tf_idf')
    )

In [19]:
%%time

STRONG_THR = 0.8

history_train = pl.concat(
    rows.with_columns(pl.col('article_id_fixed').list.len().alias('NumArticlesHistory')) \
        .explode(['article_id_fixed', 'impression_time_fixed', 'read_time_fixed', 'scroll_percentage_fixed']) \
        .sort(by=['user_id', 'impression_time_fixed']) \
        .with_columns(
            pl.col('impression_time_fixed').dt.weekday().alias('weekday'),
            pl.col('impression_time_fixed').dt.hour().alias('hour'),
        ).join(articles.select(['article_id', 'category', 'is_article_default', 'sentiment_label_int', 
                                'sentiment_score', 'entity_groups', 'topics']), 
               left_on='article_id_fixed', right_on='article_id', how='left') \
        .with_columns(
            (pl.col('sentiment_label_int') == 0).alias('is_neutral'),
            (pl.col('sentiment_label_int') == 1).alias('is_positive'),
            (pl.col('sentiment_label_int') == -1).alias('is_negative'),
            ((pl.col('sentiment_label_int') == 0) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_neutral'),
            ((pl.col('sentiment_label_int') == 1) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_positive'),
            ((pl.col('sentiment_label_int') == -1) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_negative'),
            pl.col('entity_groups').list.unique(),
        ).group_by('user_id').agg(
            pl.col('article_id_fixed'),
            pl.col('impression_time_fixed'),
            pl.col('category'),
            pl.col('NumArticlesHistory').first(),
            pl.col('read_time_fixed').median().alias('MedianReadTime'),
            pl.col('read_time_fixed').max().alias('MaxReadTime'),
            pl.col('read_time_fixed').sum().alias('TotalReadTime'),
            pl.col('scroll_percentage_fixed').median().alias('MedianScrollPercentage'),
            pl.col('scroll_percentage_fixed').max().alias('MaxScrollPercentage'),
            (pl.col('is_neutral').sum() / pl.col('NumArticlesHistory').first()).alias('NeutralPct'),
            (pl.col('is_positive').sum() / pl.col('NumArticlesHistory').first()).alias('PositivePct'),
            (pl.col('is_negative').sum() / pl.col('NumArticlesHistory').first()).alias('NegativePct'),
            (pl.col('strong_neutral').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongNeutral'),
            (pl.col('strong_positive').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongPositive'),
            (pl.col('strong_negative').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongNegative'),
            (1 - (pl.col('is_article_default').sum() / pl.col('NumArticlesHistory').first())).alias('PctNotDefaultArticles'),
            pl.col('category').mode().alias('MostFrequentCategory'),
            pl.col('weekday').mode().alias('MostFrequentWeekday'),
            pl.col('hour').mode().alias('MostFrequentHour'),
            pl.col('entity_groups').flatten(),
            pl.col('topics').flatten().alias('topics_flatten')
        ).pipe(add_topics_tf_idf_columns, topics_col='topics_flatten', vectorizer=vectorizer) \
        .drop('topics_flatten').with_columns(
            pl.col('MostFrequentCategory').list.first(),
            pl.col('MostFrequentWeekday').list.first(),
            pl.col('MostFrequentHour').list.first(),
        ).with_columns(
            (pl.col('entity_groups').list.count_matches(entity) / pl.col('NumArticlesHistory')).alias(f'{entity}Pct')
            for entity in unique_entities
        ).drop('entity_groups')
    for rows in tqdm.tqdm(history_train.iter_slices(1000), total=history_train.shape[0] // 1000)
)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
history_train = reduce_polars_df_memory_size(history_train)
history_train.head(2)

16it [00:15,  1.06it/s]                        



RAM memory GB usage = 1.238
Memory usage of dataframe is 39.38 MB
Memory usage after optimization is: 38.61 MB
Decreased by 2.0%
CPU times: user 17.8 s, sys: 1.47 s, total: 19.3 s
Wall time: 15.8 s


user_id,article_id_fixed,impression_time_fixed,category,NumArticlesHistory,MedianReadTime,MaxReadTime,TotalReadTime,MedianScrollPercentage,MaxScrollPercentage,NeutralPct,PositivePct,NegativePct,PctStrongNeutral,PctStrongPositive,PctStrongNegative,PctNotDefaultArticles,MostFrequentCategory,MostFrequentWeekday,MostFrequentHour,topics_flatten_tf_idf,PRODPct,ORGPct,LOCPct,MISCPct,EVENTPct,PERPct
u32,list[i32],list[datetime[μs]],list[i16],u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,i8,i8,list[f32],f32,f32,f32,f32,f32,f32
10623,"[9739035, 9739065, … 9770533]","[2023-04-27 14:29:44, 2023-04-27 14:29:47, … 2023-05-17 17:36:24]","[118, 414, … 140]",143,3.0,1563.0,9623.0,42.5,100.0,0.230769,0.111888,0.657343,0.097902,0.083916,0.608392,0.06993,118,3,18,"[0.016041, 0.062928, … 0.178427]",0.237762,0.888112,0.804196,0.223776,0.202797,0.874126
12016,"[9738216, 9735383, … 9769622]","[2023-04-27 07:39:48, 2023-04-27 07:41:30, … 2023-05-17 23:56:30]","[140, 118, … 140]",264,156.0,1775.0,63672.0,100.0,100.0,0.257576,0.075758,0.666667,0.106061,0.034091,0.602273,0.07197,140,5,20,"[0.009749, 0.045896, … 0.075912]",0.318182,0.852273,0.776515,0.117424,0.087121,0.787879


In [20]:
history_train.shape

(15143, 27)

# Named entity collaborative filtering

In [21]:
user_id_mapping = history_train.sort('user_id').with_row_index() \
    .select(['index', 'user_id']).rename({'index': 'user_index'})

ner_mapping = articles.select(['ner_clusters']) \
    .explode('ner_clusters') \
    .rename({'ner_clusters': 'ner'}) \
    .with_columns(pl.col('ner').str.strip_chars_start('\" ').str.to_lowercase()) \
    .unique('ner').drop_nulls().filter(pl.col('ner').str.len_chars() > 0) \
    .sort('ner').with_row_index().rename({'index': 'ner_index'})

In [22]:
ner_interactions = history_train.select(['user_id', 'article_id_fixed']).explode('article_id_fixed') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_id_fixed', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_id_fixed']) \
    .unique(['user_index', 'ner_index'])

URM_ner = sps.csr_matrix((np.ones(ner_interactions.shape[0]),
                          (ner_interactions['user_index'].to_numpy(), ner_interactions['ner_index'].to_numpy())),
                         shape=(user_id_mapping.shape[0], ner_mapping.shape[0]))

ner_interactions_test = behaviors_train.select(['user_id', 'article_ids_clicked']).explode('article_ids_clicked') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_ids_clicked', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_ids_clicked']) \
    .unique(['user_index', 'ner_index'])

URM_ner_test = sps.csr_matrix((np.ones(ner_interactions_test.shape[0]),
                               (ner_interactions_test['user_index'].to_numpy(), ner_interactions_test['ner_index'].to_numpy())),
                              shape=(user_id_mapping.shape[0], ner_mapping.shape[0]))

In [23]:
%cd /kaggle/working
! git clone https://github.com/MaurizioFD/RecSys_Course_AT_PoliMi.git

/kaggle/working


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'RecSys_Course_AT_PoliMi'...
remote: Enumerating objects: 1519, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 1519 (delta 223), reused 220 (delta 212), pack-reused 1264
Receiving objects: 100% (1519/1519), 50.33 MiB | 36.68 MiB/s, done.
Resolving deltas: 100% (909/909), done.


In [24]:
%cd /kaggle/working/RecSys_Course_AT_PoliMi

from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Evaluation.Evaluator import EvaluatorHoldout

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

item_knn = ItemKNNCFRecommender(URM_ner)
item_knn.fit(topK=100, similarity='jaccard')

evaluator = EvaluatorHoldout(URM_ner_test, cutoff_list=[10], exclude_seen=False)
result_df, _ = evaluator.evaluateRecommender(item_knn)
result_df

/kaggle/working/RecSys_Course_AT_PoliMi
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 18909 (43.2%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 43774 (100.0%), 180.52 column/sec. Elapsed time 4.04 min
EvaluatorHoldout: Ignoring 31 ( 0.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 15112 (100.0%) in 37.19 sec. Users per second: 406


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.311971,0.312913,0.030088,0.214433,0.214676,0.469902,0.319612,0.054883,0.766609,0.953152,...,0.997953,0.76504,0.997953,0.002086,5.362616,0.952586,0.016842,0.419597,3.357942,0.009295


In [25]:
pure_svd = PureSVDRecommender(URM_ner)
pure_svd.fit(num_factors=20)

result_df, _ = evaluator.evaluateRecommender(pure_svd)
result_df

PureSVDRecommender: URM Detected 1 ( 0.0%) users with no interactions.
PureSVDRecommender: URM Detected 18909 (43.2%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 4.24 sec
EvaluatorHoldout: Processed 15112 (100.0%) in 20.37 sec. Users per second: 742


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.342628,0.345141,0.048661,0.261759,0.263204,0.670397,0.392258,0.085219,0.858655,1.275579,...,0.997953,0.856898,0.997953,0.001871,5.929536,0.95823,0.015104,0.463956,3.324192,0.009294


In [26]:
%%time

def get_inview_scores(user_ners_df, recommender, f_name):
    user_index = user_ners_df['user_index'].to_list()[0]
    ners = user_ners_df['ner_index'].to_numpy()
    scores = recommender._compute_item_score([user_index], ners)[0, ners]
    return user_ners_df.with_columns(
        pl.Series(scores).alias(f_name)
    )

ner_inview_scores = behaviors_train.select(['user_id', 'article_ids_inview']).explode('article_ids_inview') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_ids_inview', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_ids_inview']) \
    .unique(['user_index', 'ner_index']) \
    .sort(['user_index', 'ner_index']).group_by('user_index').map_groups(
        lambda user_df: user_df.pipe(get_inview_scores, recommender=pure_svd, f_name='pure_svd_scores') \
            .pipe(get_inview_scores, recommender=item_knn, f_name='item_knn_scores')
    )

behaviors_ner_scores = behaviors_train.select(['impression_id', 'user_id', 'article_ids_inview']).explode('article_ids_inview') \
    .join(ner_inview_scores, on=['user_id', 'article_ids_inview'], how='left') \
    .with_columns(pl.col(['pure_svd_scores', 'item_knn_scores']).fill_null(0)) \
    .group_by(['impression_id', 'article_ids_inview']).agg(
        pl.col('pure_svd_scores').sum().alias('total_ner_svd_scores'),
        pl.col('pure_svd_scores').mean().alias('mean_ner_svd_scores'),
        pl.col('pure_svd_scores').max().alias('max_ner_svd_scores'),
        pl.col('item_knn_scores').sum().alias('total_ner_item_knn_scores'),
        pl.col('item_knn_scores').mean().alias('mean_ner_item_knn_scores'),
        pl.col('item_knn_scores').max().alias('max_ner_item_knn_scores'),
    ).with_columns(
        pl.col(['total_ner_svd_scores', 'mean_ner_svd_scores', 
                'total_ner_item_knn_scores', 'mean_ner_item_knn_scores',
                'max_ner_svd_scores', 'max_ner_item_knn_scores']).max().over('impression_id').name.suffix('_max')
    ).with_columns(
        # l1 normalization
        pl.col('total_ner_svd_scores').truediv(pl.col('total_ner_svd_scores_max')),
        pl.col('mean_ner_svd_scores').truediv(pl.col('mean_ner_svd_scores_max')),
        pl.col('max_ner_svd_scores').truediv(pl.col('max_ner_svd_scores_max')),
        pl.col('total_ner_item_knn_scores').truediv(pl.col('total_ner_item_knn_scores_max')),
        pl.col('mean_ner_item_knn_scores').truediv(pl.col('mean_ner_item_knn_scores_max')),
        pl.col('max_ner_item_knn_scores').truediv(pl.col('max_ner_item_knn_scores_max')),
    ).drop(['total_ner_svd_scores_max', 'mean_ner_svd_scores_max', 
            'max_ner_svd_scores_max', 'total_ner_item_knn_scores_max',
            'mean_ner_item_knn_scores_max', 'max_ner_item_knn_scores_max']) \
    .rename({'article_ids_inview': 'article'})
behaviors_ner_scores.head(5)

CPU times: user 3min 24s, sys: 2min 24s, total: 5min 49s
Wall time: 1min 34s


impression_id,article,total_ner_svd_scores,mean_ner_svd_scores,max_ner_svd_scores,total_ner_item_knn_scores,mean_ner_item_knn_scores,max_ner_item_knn_scores
u32,i32,f32,f32,f32,f32,f32,f32
127502926,9771903,0.081532,0.318607,0.295812,0.090677,0.347547,0.344435
276003424,9774287,-0.00004,-0.000102,0.0,0.0,0.0,0.0
543513599,8315213,-0.000027,-0.000199,-0.000054,0.0,0.0,0.0
578683966,9759544,0.015298,0.096826,0.083485,0.009548,0.066835,0.05875
52085158,9773566,0.157366,0.354297,0.653396,0.193845,0.395242,0.664962


In [27]:
del URM_ner
del URM_ner_test
del ner_inview_scores
del ner_interactions
del ner_interactions_test
gc.collect()

0

# Creating Training Dataframe

In [28]:
def normalize_endorsement(articles_endorsement_raw):
    return articles_endorsement.sort(by='impression_time').with_columns(
        (
            pl.col('endorsement_10h') / 
            pl.col('endorsement_10h').sum().over('impression_time')
        ).alias('normalized_endorsement_10h'),
        (
            pl.col('endorsement_10h') - 
            pl.col('endorsement_10h').rolling_mean(10, min_periods=1).over('article')
        ).alias('endorsement_10h_diff_rolling'),
        (
            pl.col('endorsement_10h').rolling_mean(5, min_periods=1).over('article') - 
            pl.col('endorsement_10h').rolling_mean(10, min_periods=1).over('article')
        ).alias('endorsement_macd'),
        (
            pl.col('endorsement_10h') / 
            pl.col('endorsement_10h').quantile(0.8).over('impression_time')
        ).alias('endorsement_quantile_norm_10h')
    ).with_columns(
        (
            pl.col('normalized_endorsement_10h') / 
            pl.col('normalized_endorsement_10h').rolling_max(10, min_periods=1).over('impression_time')
        ).alias('normalized_endorsement_10h_rolling_max_ratio'),
    )
    

def add_article_endorsement_feature(df_features, normalized_endorsement):
    # old return df_features.join(other=articles_endorsement, on=["article", "impression_time"], how="left")
    normalized_endorsement = normalized_endorsement.with_columns(pl.col('article').cast(pl.Int32))
    return df_features.with_columns(pl.col('impression_time').dt.round('1m').alias('rounded_impression_time'))\
                         .join(normalized_endorsement.rename({'impression_time' : 'rounded_impression_time'}), on=['article','rounded_impression_time'], how='left')\
                         .drop('rounded_impression_time')

In [29]:
def normalize_endorsement_by_article_and_user(articles_endorsement_articleuser_raw):
    return articles_endorsement_articleuser_raw.sort(by=['user_id','impression_time']).with_columns(
        (
            pl.col('endorsement_20h_articleuser') / 
            pl.col('endorsement_20h_articleuser').sum().over(['user_id','impression_time'])
        ).alias('normalized_endorsement_articleuser_20h'),
        (
            pl.col('endorsement_20h_articleuser') - 
            pl.col('endorsement_20h_articleuser').rolling_mean(10, min_periods=1).over(['article','user_id'])
        ).alias('endorsement_20h_articleuser_diff_rolling'),
        (
            pl.col('endorsement_20h_articleuser').rolling_mean(5, min_periods=1).over(['article','user_id']) - 
            pl.col('endorsement_20h_articleuser').rolling_mean(10, min_periods=1).over(['article','user_id'])
        ).alias('endorsement_articleuser_macd'),
        (
            pl.col('endorsement_20h_articleuser') / 
            pl.col('endorsement_20h_articleuser').quantile(0.8).over(['user_id','impression_time'])
        ).alias('endorsement_articleuser_quantile_norm_20h')
    ).with_columns(
        (
            pl.col('endorsement_20h_articleuser') / 
            pl.col('endorsement_20h_articleuser').rolling_max(10, min_periods=1).over(['user_id','impression_time'])
        ).alias('normalized_endorsement_articleuser_20h_rolling_max_ratio'),
    )
    

def add_article_endorsement_feature_by_article_and_user(df_features, normalized_endorsement):
    # old return df_features.join(other=articles_endorsement, on=["article", "impression_time"], how="left")
    normalized_endorsement = normalized_endorsement.with_columns(pl.col('article').cast(pl.Int32))
    return df_features.with_columns(pl.col('impression_time').dt.round('1m').alias('rounded_impression_time'))\
                         .join(normalized_endorsement.rename({'impression_time' : 'rounded_impression_time'}), on=['user_id','article','rounded_impression_time'], how='left')\
                         .drop('rounded_impression_time')

In [30]:
def add_article_endorsement_feature_leak(df_features, normalized_endorsement,period='10h'):
    # old return df_features.join(other=articles_endorsement, on=["article", "impression_time"], how="left")
    normalized_endorsement = normalized_endorsement.with_columns(pl.col('article').cast(pl.Int32)).rename({f'endorsement_{period}':f'endorsement_{period}_leak'}) \
                            .with_columns(
                                pl.col('impression_time') - pl.duration(hours=int(period[:-1]))
                            ).sort('impression_time').set_sorted('impression_time')
    
    return df_features.with_columns(pl.col('impression_time').dt.round('1m').alias('rounded_impression_time'))\
                        .sort('rounded_impression_time').set_sorted('rounded_impression_time') \
                         .join_asof(normalized_endorsement.rename({'impression_time' : 'rounded_impression_time'}),by='article', left_on='rounded_impression_time',right_on='rounded_impression_time', strategy='nearest')\
                         .drop('rounded_impression_time') \
                        .with_columns(
                            pl.col(f'endorsement_{period}_leak').fill_null(0)
                        )


In [31]:
def add_trendiness_feature_leak(df_features: pl.DataFrame, articles: pl.DataFrame, period: str = "3d"):

    topics = articles.select("topics").explode("topics").unique()
    topics = [topic for topic in topics["topics"] if topic is not None]
    # min_impression_time = df_features.select(pl.col("impression_time")).min().item()

    # topics_total_publications= articles.filter(pl.col("published_time")< min_impression_time ).select("topics") \
    # .explode("topics").group_by("topics").len()

    topics_popularity = articles.select(["published_time", "topics"]).with_columns(
        pl.col("published_time").dt.date().alias("published_date")
    ).drop("published_time").group_by("published_date").agg(
        pl.col("topics").flatten()
    ).sort("published_date").set_sorted("published_date").upsample(time_column="published_date", every="1d") \
        .rolling(index_column="published_date", period=period).agg(
        [pl.col("topics").list.count_matches(topic).sum().alias(
            f"{topic}_matches") for topic in topics]
    )

    return df_features.with_columns(
        pl.col("impression_time").dt.date().alias("impression_date")
    ).join(other=articles.select(["article_id", "topics"]), left_on="article", right_on="article_id", how="left") \
        .with_columns(
        [pl.col("topics").list.contains(topic).cast(
            pl.Int8).alias(f"{topic}_present") for topic in topics]
    ).sort('impression_date').set_sorted('impression_date') \
    .join_asof(topics_popularity.with_columns(pl.col('published_date') + pl.duration(days=1) - pl.duration(days=int(period[:-1]))) \
               .sort('published_date').set_sorted('published_date'),
               left_on="impression_date", right_on="published_date", strategy="nearest") \
        .with_columns(
        [pl.col(f"{topic}_present").mul(pl.col(f"{topic}_matches")).alias(
            f"trendiness_score_{topic}") for topic in topics]
    ).with_columns(
        pl.sum_horizontal([pl.col(f"trendiness_score_{topic}") for topic in topics]).alias(
            "trendiness_score_leak"),
    ).drop(
        [f"trendiness_score_{topic}" for topic in topics]
    ).drop(
        [f"{topic}_matches" for topic in topics]
    ).drop(
        [f"{topic}_present" for topic in topics]
    ).drop(["topics", "impression_date","published_date"])

In [32]:
%%time

NPRATIO = 2

users_mean_trendiness_scores, topics_mean_trendiness_scores = _preprocessing_history_trendiness_scores(
        history=history_train, articles=articles)
topic_mean_delays, user_mean_delays = _preprocessing_mean_delay_features(
    articles=articles, history=history_train)
windows, user_windows, user_topics_windows, user_category_windows = _preprocessing_window_features(
    history=history_train, articles=articles)
articles_endorsement = _preprocessing_article_endorsement_feature(
    behaviors=behaviors_train, period="10h")
normalized_endorsement = normalize_endorsement(articles_endorsement)
articles_endorsement_articleuser = _preprocessing_article_endorsement_feature_by_article_and_user(
    behaviors=behaviors_train, period="20h")
normalized_endorsement_articleuser = normalize_endorsement_by_article_and_user(articles_endorsement_articleuser)

topics = articles.select("topics").explode("topics").unique()
topics = [topic for topic in topics["topics"] if topic is not None]

history_counts=history_train.select(['article_id_fixed']).explode('article_id_fixed').group_by('article_id_fixed').len() \
.rename({'article_id_fixed':'article','len':'clicked_count'})

behaviors_counts=behaviors_train.select('article_ids_inview').explode('article_ids_inview').group_by('article_ids_inview').len() \
.rename({'article_ids_inview':'article','len':'inview_count'})

# dropping duplicates that can occur because of sampling strategy
train_ds = behaviors_train.pipe(create_binary_labels_column, shuffle=True, seed=123) \
    .with_columns(pl.col('gender').fill_null(2)) \
    .with_columns(pl.col('postcode').fill_null(5)) \
    .select(['impression_id', 'article_ids_inview', 'impression_time', 'labels', 
             'device_type', 'read_time', 'scroll_percentage', 'user_id', 'is_sso_user', 'gender',
             'age', 'is_subscriber', 'session_id', 'postcode']) \
    .explode(['article_ids_inview', 'labels']) \
    .rename({'article_ids_inview': 'article', 'labels': 'target'}) \
    .with_columns(pl.col('article').cast(pl.Int32)) \
    .pipe(add_trendiness_feature, articles=articles, period='1d') \
    .rename({'trendiness_score': 'trendiness_score_1d'}) \
    .pipe(add_trendiness_feature, articles=articles, period='3d') \
    .rename({'trendiness_score': 'trendiness_score_3d'}) \
    .pipe(add_trendiness_feature, articles=articles, period='5d') \
    .rename({'trendiness_score': 'trendiness_score_5d'}) \
    .pipe(add_trendiness_feature_leak, articles=articles, period='3d') \
    .rename({'trendiness_score_leak':'trendiness_score_3d_leak'}) \
    .unique(['impression_id', 'article']) \
    .with_columns(
        pl.col('impression_time').dt.weekday().alias('weekday'),
        pl.col('impression_time').dt.hour().alias('hour'),
        pl.col('article').cast(pl.Int32),
        (
            pl.col('trendiness_score_1d') / 
            pl.col('trendiness_score_3d')
        ).alias('trendiness_score_1d/3d'),
        (
            pl.col('trendiness_score_1d') / 
            pl.col('trendiness_score_5d')
        ).alias('trendiness_score_1d/5d'),
        (
            pl.col('trendiness_score_3d') / 
            pl.col('trendiness_score_3d').max().over(pl.col('impression_time').dt.date())
        ).alias('normalized_trendiness_score_overall'),
    ).join(articles.select(['article_id', 'premium', 'published_time', 'category',
                            'sentiment_score', 'sentiment_label', 'entity_groups',
                            'num_images', 'title_len', 'subtitle_len', 'body_len','num_topics',
                            'total_pageviews', 'total_inviews', 'total_read_time', 
                            'total_pageviews/inviews', 'article_type']),
           left_on='article', right_on='article_id', how='left') \
    .with_columns(
        (pl.col('impression_time') - pl.col('published_time')).dt.total_days().alias('article_delay_days'),
        (pl.col('impression_time') - pl.col('published_time')).dt.total_hours().alias('article_delay_hours'),
    ).with_columns(
        pl.col('entity_groups').list.contains(entity).alias(f'Entity_{entity}_Present')
        for entity in unique_entities
    ).drop('entity_groups') \
    .join(behaviors_ner_scores, on=['impression_id', 'article'], how='left') \
    .pipe(add_history_trendiness_scores_feature, articles=articles, 
          users_mean_trendiness_scores=users_mean_trendiness_scores,
          topics_mean_trendiness_scores=topics_mean_trendiness_scores, topics=topics) \
    .pipe(add_mean_delays_features, articles=articles,
          topic_mean_delays=topic_mean_delays, user_mean_delays=user_mean_delays) \
    .pipe(add_window_features, articles=articles, user_windows=user_windows,
          user_category_windows=user_category_windows, 
          user_topics_windows=user_topics_windows, windows=windows) \
    .pipe(add_trendiness_feature_categories, articles=articles) \
    .pipe(add_article_endorsement_feature, normalized_endorsement=normalized_endorsement) \
    .pipe(add_article_endorsement_feature_by_article_and_user, normalized_endorsement = normalized_endorsement_articleuser) \
    .pipe(add_article_endorsement_feature_leak, normalized_endorsement = normalized_endorsement) \
    .rename({
        "endorsement_10h_diff_rolling_right":"endorsement_10h_leak_diff_rolling",
        "endorsement_macd_right":"endorsement_leak_macd",
        "endorsement_quantile_norm_10h_right":"endorsement_quantile_norm_10h_leak",
        "normalized_endorsement_10h_rolling_max_ratio_right":"normalized_endorsement_10h_leak_rolling_max_ratio"
    }).join(history_counts, on='article',how='left') \
    .join(behaviors_counts , on='article', how='left')

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)

train_ds = reduce_polars_df_memory_size(train_ds)
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head()

16it [00:23,  1.46s/it]                        
2427it [00:04, 573.29it/s]                          
16it [00:01,  9.90it/s]                        
16it [00:01,  8.22it/s]                        
100%|██████████| 506/506 [00:04<00:00, 126.31it/s]



RAM memory GB usage = 3.128
Memory usage of dataframe is 1119.91 MB
Memory usage after optimization is: 661.24 MB
Decreased by 41.0%

RAM memory GB usage = 2.674
CPU times: user 10min 44s, sys: 50.1 s, total: 11min 34s
Wall time: 3min 26s


impression_id,article,user_id,impression_time,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,session_id,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,published_time,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,…,mean_user_trendiness_score,mean_topics_trendiness_score,mean_topics_mean_delay_days,mean_topics_mean_delay_hours,user_mean_delay_days,user_mean_delay_hours,window_0_history_length,window_1_history_length,window_2_history_length,window_3_history_length,is_inside_window_0,is_inside_window_1,is_inside_window_2,is_inside_window_3,window_category_score,window_topics_score,trendiness_score_category,endorsement_10h,normalized_endorsement_10h,endorsement_10h_diff_rolling,endorsement_macd,endorsement_quantile_norm_10h,normalized_endorsement_10h_rolling_max_ratio,endorsement_20h_articleuser,normalized_endorsement_articleuser_20h,endorsement_20h_articleuser_diff_rolling,endorsement_articleuser_macd,endorsement_articleuser_quantile_norm_20h,normalized_endorsement_articleuser_20h_rolling_max_ratio,endorsement_10h_leak,normalized_endorsement_10h_right,endorsement_10h_leak_diff_rolling,endorsement_leak_macd,endorsement_quantile_norm_10h_leak,normalized_endorsement_10h_leak_rolling_max_ratio,clicked_count,inview_count
u32,i32,u32,datetime[μs],i8,i8,f32,f32,bool,i8,i8,bool,u32,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,datetime[μs],i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,…,f32,f32,f32,f32,f32,f32,i16,i16,i16,i16,i8,i8,i8,i8,i16,i16,i8,u16,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,u32,u16
41650737,9767697,1260010,2023-05-18 07:00:01,1,1,38.0,null,false,2,null,false,1442549,5,50,187,238,116,4,7,0.26738,0.210084,0.363107,false,2023-05-18 06:22:53,118,0.9613,"""Negative""",5,7,2,982,3,199205,954408,2.595362e7,0.208721,…,236.066666,284.920959,3.443193,85.053078,0.0,1.6,14,1,0,6,1,0,0,0,6,11,96,14,0.041916,0.0,0.0,2.0,1.0,1,0.111111,0.0,0.0,1.0,1.0,3340,0.030563,6.7,3.1,1.56367,1.0,929,5859
579742841,9767534,362240,2023-05-18 07:00:07,0,1,88.0,null,false,2,null,false,1771620,5,29,114,169,103,4,7,0.254386,0.171598,0.221359,false,2023-05-16 20:54:50,140,0.7544,"""Negative""",1,6,21,212,5,215654,1205903,1.0432497e7,0.178832,…,192.800003,132.723724,5.302028,129.682739,0.266667,10.666667,15,0,0,0,1,0,0,0,3,3,67,3,0.008982,0.0,0.0,0.428571,0.2,1,0.030303,0.0,0.0,1.0,1.0,1009,0.012784,3.3,4.5,0.511145,0.413186,3133,2513
271334979,9417521,268275,2023-05-18 07:00:29,0,1,31.0,100.0,false,2,null,false,1770914,5,7,33,44,32,4,7,0.212121,0.159091,0.064078,true,2022-09-12 17:50:50,565,0.7289,"""Neutral""",2,7,15,654,2,null,null,null,null,…,282.16394,191.472092,84.034439,2022.176392,3.016393,76.52459,90,33,5,62,1,0,0,0,0,2,1,1,0.002994,0.0,0.0,0.142857,0.5,1,0.047619,0.0,0.0,1.0,1.0,331,0.003986,3.8,2.6,0.171592,0.495509,81,4051
550481897,9657528,2039130,2023-05-18 07:00:27,0,1,98.0,100.0,false,2,null,false,1679074,5,7,17,66,12,4,7,0.411765,0.106061,0.03301,false,2023-03-02 14:49:58,414,0.7882,"""Positive""",1,4,11,233,2,72719,325517,3.620928e6,0.223395,…,237.067413,270.067261,19.106453,461.467926,11.719101,283.651672,55,34,21,20,1,0,0,0,7,0,49,1,0.002994,0.0,0.0,0.142857,0.333333,1,0.05,0.0,0.0,1.0,1.0,9,0.000079,-1.8,1.8,0.00455,0.00536,80,20
346943718,9514481,166553,2023-05-18 07:00:10,0,2,30.0,null,false,2,null,false,643203,5,8,37,49,34,4,7,0.216216,0.163265,0.071845,true,2022-11-16 11:15:32,414,0.9501,"""Neutral""",7,9,30,371,3,null,null,null,null,…,178.580643,180.074875,89.710144,2158.245117,1.147465,29.62212,141,75,21,1,1,0,0,0,10,9,49,1,0.002994,0.0,0.0,0.142857,0.333333,1,0.066667,0.0,0.0,1.0,1.0,406,0.004889,5.7,3.5,0.210472,0.225681,103,7322


In [33]:
train_ds.shape

(2585747, 89)

## Session Features

In [34]:
# behaviors_train.select(['session_id', 'impression_time', 'article_ids_clicked']) \
#     .group_by('session_id').map_groups(
#         lambda group: group.explode('article_ids_clicked').sort('impression_time') \
#             .select(
#                 pl.col('session_id').first(),
#                 pl.col('article_ids_clicked').first().alias('article_id'),
#                 (pl.col('impression_time').last() - pl.col('impression_time').first()).dt.total_minutes().alias('induced_session_len'),
#                 (pl.col('impression_time').count() - 1).alias('num_clicked_articles_following')
#             )
#     ).group_by('article_id').agg(
#         pl.col('num_clicked_articles_following').mean().alias('mean_num_clicked_articles_following'),
#         pl.col('induced_session_len').mean().alias('mean_induced_session_len'),
#     )

In [35]:
%%time

last_history_df = history_train.with_columns(
    pl.col('impression_time_fixed').list.max().alias('last_history_impression_time'),
    pl.col('article_id_fixed').list.tail(1).alias('last_history_article'),
).select(['user_id', 'last_history_impression_time', 'last_history_article'])

last_session_time_df = behaviors_train.select(['session_id', 'user_id', 'impression_time', 'article_ids_inview']) \
    .group_by('session_id').agg(
        pl.col('user_id').first(), 
        pl.col('impression_time').max().alias('session_time'), 
        pl.col('article_ids_inview').flatten().alias('all_seen_articles'),
        (pl.col('impression_time').max() - pl.col('impression_time').min()).dt.total_minutes().alias('session_duration'),
    ).with_columns(
        pl.col(['session_time', 'session_duration']).shift(1).over('user_id').name.prefix('last_'),
        pl.col('all_seen_articles').list.unique().shift(1).over('user_id'),
        pl.col('session_duration').rolling_mean(100, min_periods=1).over('user_id').alias('mean_prev_sessions_duration'),
    ).with_columns(pl.col(['last_session_duration']).fill_null(0)) \
    .join(last_history_df, on='user_id', how='left') \
    .with_columns(
        pl.col('last_session_time').fill_null(pl.col('last_history_impression_time')),
        pl.col('all_seen_articles').fill_null(pl.col('last_history_article')),
    ).select(['session_id', 'last_session_time', 'last_session_duration', 
              'all_seen_articles', 'mean_prev_sessions_duration'])
    
gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
last_session_time_df.head(2)


RAM memory GB usage = 2.662
CPU times: user 1.48 s, sys: 114 ms, total: 1.59 s
Wall time: 1.18 s


session_id,last_session_time,last_session_duration,all_seen_articles,mean_prev_sessions_duration
u32,datetime[μs],i64,list[i32],f64
979075,2023-05-17 21:12:27,0,[8922242],0.0
669488,2023-05-17 10:47:38,0,[9769804],0.0


In [36]:
train_ds = train_ds.join(last_session_time_df, on='session_id', how='left').with_columns(
    (pl.col('impression_time') - pl.col('last_session_time')).dt.total_hours().alias('last_session_time_hour_diff'),
    ((pl.col('last_session_time') - pl.col('published_time')).dt.total_hours() > 0).alias('is_new_article'),
    pl.col('all_seen_articles').list.contains(pl.col('article')).alias('is_already_seen_article'),
).drop(['published_time', 'session_id', 'all_seen_articles', 'last_session_time'])

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head(2)


RAM memory GB usage = 3.045


impression_id,article,user_id,impression_time,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,…,user_mean_delay_hours,window_0_history_length,window_1_history_length,window_2_history_length,window_3_history_length,is_inside_window_0,is_inside_window_1,is_inside_window_2,is_inside_window_3,window_category_score,window_topics_score,trendiness_score_category,endorsement_10h,normalized_endorsement_10h,endorsement_10h_diff_rolling,endorsement_macd,endorsement_quantile_norm_10h,normalized_endorsement_10h_rolling_max_ratio,endorsement_20h_articleuser,normalized_endorsement_articleuser_20h,endorsement_20h_articleuser_diff_rolling,endorsement_articleuser_macd,endorsement_articleuser_quantile_norm_20h,normalized_endorsement_articleuser_20h_rolling_max_ratio,endorsement_10h_leak,normalized_endorsement_10h_right,endorsement_10h_leak_diff_rolling,endorsement_leak_macd,endorsement_quantile_norm_10h_leak,normalized_endorsement_10h_leak_rolling_max_ratio,clicked_count,inview_count,last_session_duration,mean_prev_sessions_duration,last_session_time_hour_diff,is_new_article,is_already_seen_article
u32,i32,u32,datetime[μs],i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,…,f32,i16,i16,i16,i16,i8,i8,i8,i8,i16,i16,i8,u16,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,u32,u16,i64,f64,i64,bool,bool
41650737,9767697,1260010,2023-05-18 07:00:01,1,1,38.0,null,false,2,null,false,5,50,187,238,116,4,7,0.26738,0.210084,0.363107,false,118,0.9613,"""Negative""",5,7,2,982,3,199205,954408,2.595362e7,0.208721,"""article_default""",0,…,1.6,14,1,0,6,1,0,0,0,6,11,96,14,0.041916,0.0,0.0,2.0,1.0,1,0.111111,0.0,0.0,1.0,1.0,3340,0.030563,6.7,3.1,1.56367,1.0,929,5859,0,0.0,-167,true,false
579742841,9767534,362240,2023-05-18 07:00:07,0,1,88.0,null,false,2,null,false,5,29,114,169,103,4,7,0.254386,0.171598,0.221359,false,140,0.7544,"""Negative""",1,6,21,212,5,215654,1205903,1.0432497e7,0.178832,"""article_default""",1,…,10.666667,15,0,0,0,1,0,0,0,3,3,67,3,0.008982,0.0,0.0,0.428571,0.2,1,0.030303,0.0,0.0,1.0,1.0,1009,0.012784,3.3,4.5,0.511145,0.413186,3133,2513,0,1.0,0,true,false


In [37]:
train_ds.select(pl.col('is_new_article').sum() / train_ds.shape[0], pl.col('is_already_seen_article').sum() / train_ds.shape[0])

is_new_article,is_already_seen_article
f64,f64
0.559822,0.023945


In [38]:
del last_history_df
del last_session_time_df
gc.collect()

0

## Popularities

In [39]:
%%time

articles_date_popularity = articles.select(['published_time', 'article_id']) \
    .group_by(pl.col('published_time').dt.date().alias('published_date')) \
    .agg(pl.col('article_id').count().alias('daily_articles')) \

published_category_popularity = articles.select(['published_time', 'article_id', 'category']) \
    .group_by([pl.col('published_time').dt.date().alias('published_date'), 'category']) \
    .agg(pl.col('article_id').count().alias('category_daily_articles')) \
    .join(articles_date_popularity, on='published_date', how='left') \
    .with_columns((pl.col('category_daily_articles') / pl.col('daily_articles')).alias('category_daily_pct')) \
    .drop(['category_daily_articles', 'daily_articles'])

train_ds = train_ds.join(published_category_popularity, how='left', right_on=['published_date', 'category'],
                         left_on=[pl.col('impression_time').dt.date() - pl.duration(days=1), 'category']) \
    .rename({'category_daily_pct': 'yesterday_category_daily_pct'}) \
    .with_columns(pl.col('yesterday_category_daily_pct').fill_null(0))
train_ds = reduce_polars_df_memory_size(train_ds)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head(2)

Memory usage of dataframe is 701.62 MB
Memory usage after optimization is: 649.53 MB
Decreased by 7.4%

RAM memory GB usage = 3.092
CPU times: user 1.4 s, sys: 410 ms, total: 1.81 s
Wall time: 1.62 s


impression_id,article,user_id,impression_time,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,…,window_0_history_length,window_1_history_length,window_2_history_length,window_3_history_length,is_inside_window_0,is_inside_window_1,is_inside_window_2,is_inside_window_3,window_category_score,window_topics_score,trendiness_score_category,endorsement_10h,normalized_endorsement_10h,endorsement_10h_diff_rolling,endorsement_macd,endorsement_quantile_norm_10h,normalized_endorsement_10h_rolling_max_ratio,endorsement_20h_articleuser,normalized_endorsement_articleuser_20h,endorsement_20h_articleuser_diff_rolling,endorsement_articleuser_macd,endorsement_articleuser_quantile_norm_20h,normalized_endorsement_articleuser_20h_rolling_max_ratio,endorsement_10h_leak,normalized_endorsement_10h_right,endorsement_10h_leak_diff_rolling,endorsement_leak_macd,endorsement_quantile_norm_10h_leak,normalized_endorsement_10h_leak_rolling_max_ratio,clicked_count,inview_count,last_session_duration,mean_prev_sessions_duration,last_session_time_hour_diff,is_new_article,is_already_seen_article,yesterday_category_daily_pct
u32,i32,u32,date,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,…,i16,i16,i16,i16,i8,i8,i8,i8,i16,i16,i8,u16,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,u32,u16,i8,f32,i16,bool,bool,f32
41650737,9767697,1260010,2023-05-17,1,1,38.0,null,false,2,null,false,5,50,187,238,116,4,7,0.26738,0.210084,0.363107,false,118,0.9613,"""Negative""",5,7,2,982,3,199205,954408,2.595362e7,0.208721,"""article_default""",0,…,14,1,0,6,1,0,0,0,6,11,96,14,0.041916,0.0,0.0,2.0,1.0,1,0.111111,0.0,0.0,1.0,1.0,3340,0.030563,6.7,3.1,1.56367,1.0,929,5859,0,0.0,-167,true,false,0.222222
579742841,9767534,362240,2023-05-17,0,1,88.0,null,false,2,null,false,5,29,114,169,103,4,7,0.254386,0.171598,0.221359,false,140,0.7544,"""Negative""",1,6,21,212,5,215654,1205903,1.0432497e7,0.178832,"""article_default""",1,…,15,0,0,0,1,0,0,0,3,3,67,3,0.008982,0.0,0.0,0.428571,0.2,1,0.030303,0.0,0.0,1.0,1.0,1009,0.012784,3.3,4.5,0.511145,0.413186,3133,2513,0,1.0,0,true,false,0.214286


# Adding history features

In [40]:
def list_pct_matches_with_col(a, b) -> pl.Expr:
    '''
    Returns an expression to count the percentage of matching element in a list with another column.
    The polars function count_matches cannot be used since it wants only a single element, 
    variable element from row to row.
    '''
    return pl.when(pl.col(a).list.len() == 0).then(0.0) \
        .otherwise((pl.col(a).list.len() - (pl.col(a).list.set_difference(pl.col(b))).list.len()) / pl.col(a).list.len())


def list_pct_matches_with_constant(a, value) -> pl.Expr:
    '''
    Returns an expression to count the percentage of matching element in a list with a constant value.
    The polars function count_matches cannot be used since it wants only a single element, 
    variable element from row to row.
    '''
    return pl.when(pl.col(a).list.len() == 0).then(0.0) \
        .otherwise(pl.col(a).list.count_matches(value) / pl.col(a).list.len())


def cosine_similarity(x, y):
    x = np.array(x)
    y = np.array(y)
    normalization = np.linalg.norm(x, 2) * np.linalg.norm(y, 2)
    return np.dot(x, y) / normalization if normalization > 0 else 0


prev_train_columns = [c for c in train_ds.columns if c not in ['impression_id', 'article', 'impression_time']]

train_ds = pl.concat(
    rows.join(history_train.select(['user_id', 'article_id_fixed', 'impression_time_fixed']), on='user_id', how='left') \
        .join(articles.select(['article_id', 'topics', 'entity_groups', 'topics_idf'] + topic_model_columns), 
              left_on='article', right_on='article_id', how='left') \
        .explode(['article_id_fixed', 'impression_time_fixed']) \
        .join(articles.select(['article_id', 'topics', 'entity_groups'] + topic_model_columns), 
              left_on='article_id_fixed', right_on='article_id', how='left') \
        .rename({'topics_right': 'topics_history', 'entity_groups_right': 'entity_groups_history'}) \
        .rename({f'topic_model_embedding_{i}_right': f'topic_model_embedding_{i}_history' for i in range(N_COMPONENTS)}) \
        .with_columns(
            # 1/delay gives the weight for the weighted mean of the lda embeddings
            (1 / (pl.col('impression_time') - pl.col('impression_time_fixed')).dt.total_hours().sqrt()).alias('history_weight'),
            (pl.col("topics").list.set_intersection(pl.col("topics_history")).list.len().truediv(
                pl.col("topics").list.set_union(pl.col("topics_history")).list.len()
            )).alias("JS"),
            pl.col('entity_groups').list.set_intersection(pl.col('entity_groups_history')).list.len().alias('common_entities'),
        ).drop(['entity_groups_history', 'entity_groups', 'topics', 'topics_history']) \
        .with_columns(
            # summing delays to normalize them before weighted mean
            pl.col('history_weight').sum().over(['impression_id', 'article']).alias('history_weight_sum'),
            *[pl.col(x).mul(pl.col(f'{x}_history')).alias(f'{x}_dot') for x in topic_model_columns],
            *[pl.col(x).mul(pl.col(x)) for x in topic_model_columns],
            *[pl.col(f'{x}_history').mul(pl.col(f'{x}_history')).alias(f'{x}_history_square') for x in topic_model_columns],
        ).with_columns(
            # weights now sum to 1
            pl.col('history_weight').truediv(pl.col('history_weight_sum')),
            pl.sum_horizontal(topic_model_columns).sqrt().alias('topic_model_norm'),
            pl.sum_horizontal([f'{x}_history_square' for x in topic_model_columns]).sqrt().alias('topic_model_history_norm'),
            pl.sum_horizontal([f'{x}_dot' for x in topic_model_columns]).alias('topic_model_dot'),
        ).with_columns(
            *[pl.col(f'{x}_history').mul(pl.col('history_weight')).alias(f'{x}_history_weighted') for i, x in enumerate(topic_model_columns)],
            pl.col('topic_model_dot').truediv(pl.col('topic_model_norm').mul('topic_model_history_norm')).alias('topic_model_cosine')
        ).group_by(['impression_id', 'article']).agg(
            pl.col(prev_train_columns).first(),
            pl.col('topics_idf').first(),
            pl.col('common_entities').mean().alias('MeanCommonEntities'),
            pl.col('common_entities').max().alias('MaxCommonEntities'),
            pl.col("JS").mean().alias("mean_JS"),
            pl.col("JS").min().alias("min_JS"),
            pl.col("JS").max().alias("max_JS"),
            pl.col("JS").std().alias("std_JS"),
            pl.col("JS").mul(pl.col("history_weight")).sum().alias("weighted_mean_JS"),
            pl.col("topic_model_cosine").mean().alias("mean_topic_model_cosine"),
            pl.col("topic_model_cosine").min().alias("min_topic_model_cosine"),
            pl.col("topic_model_cosine").max().alias("max_topic_model_cosine"),
            pl.col("topic_model_cosine").std().alias("std_topic_model_cosine"),
            pl.col("topic_model_cosine").mul(pl.col("history_weight")).sum().alias("weighted_mean_topic_model_cosine"),
            *[pl.col(f'{x}_history').mean().alias(f'lda_{i}_history_mean') for i, x in enumerate(topic_model_columns)],
            *[pl.col(f'{x}_history_weighted').sum().alias(f'lda_{i}_history_weighted_mean') for i, x in enumerate(topic_model_columns)]
        ).join(history_train.drop(['article_id_fixed', 'impression_time_fixed']), on='user_id', how='left') \
        .with_columns(
            pl.struct(['topics_idf', 'topics_flatten_tf_idf']).map_elements(
                lambda x: cosine_similarity(x['topics_idf'], x['topics_flatten_tf_idf']), return_dtype=pl.Float64
            ).cast(pl.Float32).alias('topics_cosine'),
            (pl.col('category') == pl.col('MostFrequentCategory')).alias('IsFavouriteCategory'),
            pl.col('category_right').list.n_unique().alias('NumberDifferentCategories'),
            list_pct_matches_with_col('category_right', 'category').alias('PctCategoryMatches'),
        ).with_columns(
            [list_pct_matches_with_constant('category_right', c).alias(f'Category_{c_str}_Pct')
             for c, c_str in unique_categories.items()]
        ).drop(['topics_idf', 'topics_flatten', 'topics_flatten_tf_idf', 'category_right'])
    for rows in tqdm.tqdm(train_ds.iter_slices(10000), total=train_ds.shape[0] // 10000)
)
train_ds = reduce_polars_df_memory_size(train_ds)

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head(2)

259it [31:27,  7.29s/it]


Memory usage of dataframe is 1700.14 MB
Memory usage after optimization is: 1317.92 MB
Decreased by 22.5%

RAM memory GB usage = 5.047


impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,MostFrequentWeekday,MostFrequentHour,PRODPct,ORGPct,LOCPct,MISCPct,EVENTPct,PERPct,topics_cosine,IsFavouriteCategory,NumberDifferentCategories,PctCategoryMatches,Category_underholdning_Pct,Category_ferie_Pct,Category_bibliotek_Pct,Category_penge_Pct,Category_side9_Pct,Category_krimi_Pct,Category_forbrug_Pct,Category_auto_Pct,Category_dagsorden_Pct,Category_musik_Pct,Category_haandvaerkeren_Pct,Category_podcast_Pct,Category_plus_Pct,Category_horoskoper_Pct,Category_vin_Pct,Category_biler_Pct,Category_incoming_Pct,Category_services_Pct,Category_nyheder_Pct,Category_sport_Pct,Category_om_ekstra_bladet_Pct,Category_opinionen_Pct,Category_nationen_Pct,Category_video_Pct,Category_sex_og_samliv_Pct
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,i8,i8,f32,f32,f32,f32,f32,f32,f32,bool,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
414477177,9142564,1221334,0,1,20.0,null,false,2,null,false,5,56,184,268,180,4,7,0.304348,0.208955,0.357282,true,414,0.955,"""Neutral""",4,5,23,868,4,null,null,null,null,"""article_default""",446,10725,…,7,8,0.246377,0.942029,0.869565,0.26087,0.123188,0.833333,0.107609,false,9,0.942029,0.101449,0.0,0.0,0.014493,0.0,0.130435,0.028986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007246,0.0,0.0,0.543478,0.123188,0.0,0.0,0.036232,0.0,0.014493
202974908,9770867,1085569,0,2,35.0,null,false,2,null,false,5,25,131,203,104,4,7,0.19084,0.123153,0.254369,false,118,0.723,"""Negative""",1,6,16,305,4,74198,349524,6.149025e6,0.212283,"""article_default""",0,2,…,6,3,0.275081,0.932039,0.815534,0.210356,0.174757,0.854369,0.210324,true,8,0.977346,0.226537,0.0,0.0,0.006472,0.0,0.236246,0.019417,0.0,0.0,0.035599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.349515,0.110032,0.0,0.0,0.016181,0.0,0.0


In [41]:
NORMALIZE_OVER_IMPRESSION_ID = [
    'trendiness_score_3d','trendiness_score_5d', 'endorsement_10h', 
    'total_pageviews/inviews', 'mean_JS','mean_topic_model_cosine', 'topics_cosine',
    'article_delay_hours', 'total_pageviews','total_inviews', 'trendiness_score_category',
    'std_JS', 'total_read_time', 'endorsement_10h_leak',"trendiness_score_3d_leak",
    "clicked_count","inview_count",
]
RANK_IMPRESSION_DESCENDING = [
    'trendiness_score_3d','trendiness_score_5d', 'endorsement_10h', 
    'total_pageviews/inviews', 'mean_JS', 'mean_topic_model_cosine', 'topics_cosine',
    'total_pageviews', 'total_read_time','total_inviews', 'trendiness_score_category',
    'endorsement_10h_leak',"trendiness_score_3d_leak","clicked_count","inview_count"
]
RANK_IMPRESSION_ASCENDING = [
    'article_delay_hours', 'std_JS', 'mean_topics_mean_delay_hours'
]
NORMALIZE_OVER_USER_ID = [
    'mean_JS', 'std_JS', 'mean_topic_model_cosine', 'topics_cosine',
]
NORMALIZE_OVER_ARTICLE = [
    'article_delay_hours', 'mean_JS', 'std_JS', 'mean_topic_model_cosine', 'topics_cosine',
]
LIST_DIVERSITY = [
    'category', 'sentiment_label', 'article_type'
]
NORMALIZE_OVER_ARTICLE_AND_USER_ID= ['endorsement_20h_articleuser']

expressions = [
    [(pl.col(c) / pl.col(c).max().over(pl.col('impression_id'))).alias(f'{c}_l_inf_impression')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [pl.col(c).rank(method='min').over(pl.col('impression_id')).alias(f'{c}_rank_impression')
     for c in RANK_IMPRESSION_ASCENDING],
    [pl.col(c).rank(method='min', descending=True).over(pl.col('impression_id')).alias(f'{c}_rank_impression')
     for c in RANK_IMPRESSION_DESCENDING],
    [pl.col(c).std().over(pl.col('impression_id')).alias(f'std_impression_{c}')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [pl.col(c).skew().over(pl.col('impression_id')).alias(f'skew_impression_{c}')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [pl.col(c).kurtosis().over(pl.col('impression_id')).alias(f'kurtosis_impression_{c}')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [pl.col(c).entropy().over(pl.col('impression_id')).alias(f'entropy_impression_{c}')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [(pl.col(c) - pl.col(c).median().over(pl.col('impression_id'))).alias(f'{c}_minus_median_impression')
     for c in NORMALIZE_OVER_IMPRESSION_ID],
    [(pl.col(c) / pl.col(c).max().over(pl.col('user_id'))).alias(f'{c}_l_inf_user')
     for c in NORMALIZE_OVER_USER_ID],
    [(pl.col(c) / pl.col(c).max().over(pl.col('article'))).alias(f'{c}_l_inf_article')
     for c in NORMALIZE_OVER_ARTICLE],
    [pl.col(c).n_unique().over('impression_id').alias(f'{c}_diversity_impression')
     for c in LIST_DIVERSITY],
    [(pl.col(c) / pl.col(c).max().over([pl.col('article'),pl.col('user_id')])).alias(f'{c}_l_inf_article_user')
     for c in NORMALIZE_OVER_ARTICLE_AND_USER_ID],
]
    
for expressions_group in tqdm.tqdm(expressions):
    train_ds = train_ds.with_columns(expressions_group)
    train_ds = reduce_polars_df_memory_size(train_ds)

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
train_ds.head(2)

  0%|          | 0/12 [00:00<?, ?it/s]

Memory usage of dataframe is 1595.65 MB


  8%|▊         | 1/12 [00:05<00:59,  5.39s/it]

Memory usage after optimization is: 1487.14 MB
Decreased by 6.8%
Memory usage of dataframe is 1517.04 MB


 17%|█▋        | 2/12 [00:08<00:42,  4.25s/it]

...

 92%|█████████▏| 11/12 [01:03<00:04,  4.04s/it]

Memory usage after optimization is: 2519.34 MB
Decreased by 0.9%
Memory usage of dataframe is 2539.07 MB


100%|██████████| 12/12 [01:06<00:00,  5.50s/it]

Memory usage after optimization is: 2529.21 MB
Decreased by 0.4%

RAM memory GB usage = 5.303


impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_total_read_time,entropy_impression_endorsement_10h_leak,entropy_impression_trendiness_score_3d_leak,entropy_impression_clicked_count,entropy_impression_inview_count,trendiness_score_3d_minus_median_impression,trendiness_score_5d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,total_read_time_minus_median_impression,endorsement_10h_leak_minus_median_impression,trendiness_score_3d_leak_minus_median_impression,clicked_count_minus_median_impression,inview_count_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,endorsement_20h_articleuser_l_inf_article_user
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u8,u8,f32
414477177,9142564,1221334,0,1,20.0,null,false,2,null,false,5,56,184,268,180,4,7,0.304348,0.208955,0.357282,true,414,0.955,"""Neutral""",4,5,23,868,4,null,null,null,null,"""article_default""",446,10725,…,NaN,2.362862,1.837803,2.192893,2.161508,2.023523,1.634897,81.0,122.0,-30.5,null,-0.011124,-0.012855,-0.046881,10168.5,null,null,-18.0,-0.010008,null,-323.0,106.0,-353.0,3139.5,0.176207,0.260702,0.84489,0.139595,0.984577,0.116994,0.343315,0.80143,0.149023,7,3,2,0.8
202974908,9770867,1085569,0,2,35.0,null,false,2,null,false,5,25,131,203,104,4,7,0.19084,0.123153,0.254369,false,118,0.723,"""Negative""",1,6,16,305,4,74198,349524,6.149025e6,0.212283,"""article_default""",0,2,…,NaN,2.74979,2.604283,2.536428,2.554331,2.537132,1.649724,13.0,0.0,19.0,0.012373,-0.017599,-0.063457,-0.057305,-8.0,-11874.5,-155330.0,0.0,0.031869,1431671.5,207.0,15.0,-116.0,43.0,0.314845,0.637916,0.823828,0.294911,0.012903,0.132094,0.397147,0.815346,0.234787,6,3,2,1.0


# Training

In [42]:
# subsample dataset

behaviors_subsample = behaviors_train.select(['impression_id','user_id','article_ids_inview','article_ids_clicked']) \
    .pipe(
        sampling_strategy_wu2019, npratio=NPRATIO, shuffle=False, with_replacement=True, seed=123
    ).explode('article_ids_inview').drop('article_ids_clicked').rename({'article_ids_inview' : 'article'}) \
    .with_columns(pl.col('user_id').cast(pl.UInt32), pl.col('article').cast(pl.Int32)) \
    .select(['impression_id', 'user_id', 'article'])

train_ds = behaviors_subsample.join(train_ds, on=['impression_id','user_id','article'], how='left')
train_ds.head()

impression_id,user_id,article,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_total_read_time,entropy_impression_endorsement_10h_leak,entropy_impression_trendiness_score_3d_leak,entropy_impression_clicked_count,entropy_impression_inview_count,trendiness_score_3d_minus_median_impression,trendiness_score_5d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,total_read_time_minus_median_impression,endorsement_10h_leak_minus_median_impression,trendiness_score_3d_leak_minus_median_impression,clicked_count_minus_median_impression,inview_count_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,endorsement_20h_articleuser_l_inf_article_user
u32,u32,i32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u8,u8,f32
149474,139836,9778728,0,2,13.0,null,false,2,null,false,5,150,521,836,419,3,7,0.287908,0.179426,0.880068,false,142,0.9654,"""Negative""",1,5,18,251,7,22415,220247,1.004828e6,0.101772,"""article_default""",0,0,…,1.612282,1.739272,1.558951,1.774823,1.725289,-0.0,1.76343,281.5,447.5,-873.5,-0.111306,-0.005177,0.107542,-0.02667,-1.0,-62463.5,-165503.0,4.0,-0.033664,-4.981345e6,5.0,165.5,null,-886.0,0.447581,0.403328,0.974183,0.36119,0.0,0.233531,0.335164,0.899841,0.301919,4,3,1,1.0
149474,139836,9778669,0,2,13.0,null,false,2,null,false,5,85,199,313,266,3,7,0.427136,0.271565,0.336149,false,118,0.9481,"""Negative""",1,5,11,150,4,74491,373488,4.365609e6,0.199447,"""article_default""",0,1,…,1.612282,1.739272,1.558951,1.774823,1.725289,-0.0,1.76343,-40.5,-75.5,179.5,-0.013631,0.003967,-0.063773,0.015762,0.0,-10387.5,-12262.0,0.0,0.028015,-1.620564e6,-397.0,12.5,null,-56.0,0.501163,0.632488,0.771845,0.437299,0.043478,0.266627,0.511387,0.774927,0.319083,4,3,1,1.0
149474,139836,9778657,1,2,13.0,null,false,2,null,false,5,45,117,183,138,3,7,0.384615,0.245902,0.197635,false,118,0.8347,"""Neutral""",2,6,31,336,3,108389,478098,7.606737e6,0.226709,"""article_default""",0,1,…,1.612282,1.739272,1.558951,1.774823,1.725289,-0.0,1.76343,-122.5,-205.5,368.5,0.013631,0.05309,-0.030841,0.218611,0.0,23510.5,92348.0,0.0,0.051035,1.620564e6,-5.0,-115.5,null,276.0,0.78901,0.718014,0.810741,0.801144,0.066667,0.350947,0.730242,0.805005,0.604487,4,3,1,1.0
150528,143471,9778682,0,2,25.0,null,false,2,null,false,5,69,206,334,201,3,7,0.334951,0.206587,0.347973,false,498,0.9546,"""Negative""",1,5,20,267,3,143520,455723,9.298546e6,0.314928,"""article_default""",0,1,…,1.924178,2.126514,2.008478,2.181736,2.013598,-0.0,2.173429,7.0,21.0,204.0,0.088219,0.013219,0.085925,-0.001332,1.0,48254.0,82235.0,-47.0,0.013249,4.223417e6,160.0,0.0,null,498.

In [43]:
train_ds = train_ds.drop(['impression_id', 'article', 'user_id']).to_pandas()
X = train_ds.drop(columns='target')

bool_columns = [f'Entity_{entity}_Present' for entity in unique_entities]
bool_columns += ['is_subscriber', 'is_sso_user', 'premium', 'is_new_article', 'is_already_seen_article', 'IsFavouriteCategory']
string_columns = ['sentiment_label', 'article_type']
categorical_int_columns = ['device_type', 'gender', 'weekday', 'category', 'MostFrequentCategory', 'MostFrequentWeekday', 'postcode']
categorical_columns = bool_columns + string_columns + categorical_int_columns
numerical_columns = [c for c in train_ds.columns if c not in categorical_columns + ['target']]

X[numerical_columns] = X[numerical_columns].replace([-np.inf, np.inf], np.nan)
min_values = {}
for col in numerical_columns:
    min_values[col] = X[col].min()
    X[col] = X[col].fillna(min_values[col])
X_numerical = X[numerical_columns].values
scaler = MinMaxScaler()
X_numerical = scaler.fit_transform(X_numerical)

categorical_encoder = OneHotEncoder(handle_unknown='ignore', drop='if_binary', sparse_output=False)
X_categorical = categorical_encoder.fit_transform(X[categorical_columns])

X = np.concatenate([X_numerical, X_categorical], axis=1)
y = train_ds['target'].values

In [44]:
del train_ds
del X_categorical

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 10.06


In [45]:
model = FastRGFClassifier(n_estimators=5000, max_depth=6, max_leaf=100, l1=500.0, 
                          max_bin=256, l2=10000.0, n_jobs=2)

model.fit(X, y)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 9.394


In [46]:
del history_train
del behaviors_train

del users_mean_trendiness_scores, topics_mean_trendiness_scores
del topic_mean_delays, user_mean_delays
del windows, user_windows, user_topics_windows, user_category_windows
del articles_endorsement
del normalized_endorsement
del articles_endorsement_articleuser
del normalized_endorsement_articleuser
del history_counts, behaviors_counts

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 9.132


# Model Evaluation

In [47]:
user_id_mapping = history_val.sort('user_id').with_row_index() \
    .select(['index', 'user_id']).rename({'index': 'user_index'})

ner_interactions = history_val.select(['user_id', 'article_id_fixed']).explode('article_id_fixed') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_id_fixed', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_id_fixed']) \
    .unique(['user_index', 'ner_index'])

URM_ner = sps.csr_matrix((np.ones(ner_interactions.shape[0]),
                          (ner_interactions['user_index'].to_numpy(), ner_interactions['ner_index'].to_numpy())),
                         shape=(user_id_mapping.shape[0], ner_mapping.shape[0]))

ner_interactions_test = behaviors_val.select(['user_id', 'article_ids_clicked']).explode('article_ids_clicked') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_ids_clicked', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_ids_clicked']) \
    .unique(['user_index', 'ner_index'])

URM_ner_test = sps.csr_matrix((np.ones(ner_interactions_test.shape[0]),
                               (ner_interactions_test['user_index'].to_numpy(), ner_interactions_test['ner_index'].to_numpy())),
                              shape=(user_id_mapping.shape[0], ner_mapping.shape[0]))

In [48]:
item_knn = ItemKNNCFRecommender(URM_ner)
item_knn.fit(topK=100, similarity='jaccard')

evaluator = EvaluatorHoldout(URM_ner_test, cutoff_list=[10], exclude_seen=False)
result_df, _ = evaluator.evaluateRecommender(item_knn)
result_df

ItemKNNCFRecommender: URM Detected 3 ( 0.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 19035 (43.5%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 43774 (100.0%), 179.60 column/sec. Elapsed time 4.06 min
EvaluatorHoldout: Ignoring 32 ( 0.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 15310 (100.0%) in 40.94 sec. Users per second: 374


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.338445,0.339461,0.033197,0.241643,0.241977,0.493913,0.346767,0.060464,0.784651,1.033204,...,0.997914,0.783014,0.997914,0.002568,5.473111,0.953841,0.020621,0.428217,3.43338,0.010093


In [49]:
pure_svd = PureSVDRecommender(URM_ner)
pure_svd.fit(num_factors=20)

result_df, _ = evaluator.evaluateRecommender(pure_svd)
result_df

PureSVDRecommender: URM Detected 3 ( 0.0%) users with no interactions.
PureSVDRecommender: URM Detected 19035 (43.5%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 4.27 sec
EvaluatorHoldout: Processed 15310 (100.0%) in 22.69 sec. Users per second: 675


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.365382,0.368539,0.050795,0.283615,0.285707,0.695447,0.415659,0.089191,0.876486,1.342637,...,0.997914,0.874658,0.997914,0.001527,5.728555,0.954883,0.012263,0.448203,3.536612,0.010003


In [50]:
%%time

def get_inview_scores(user_ners_df, recommender, f_name):
    user_index = user_ners_df['user_index'].to_list()[0]
    ners = user_ners_df['ner_index'].to_numpy()
    scores = recommender._compute_item_score([user_index], ners)[0, ners]
    return user_ners_df.with_columns(
        pl.Series(scores).alias(f_name)
    )

ner_inview_scores = behaviors_val.select(['user_id', 'article_ids_inview']).explode('article_ids_inview') \
    .join(articles.select(['article_id', 'ner_clusters']), left_on='article_ids_inview', right_on='article_id', how='left') \
    .explode('ner_clusters').unique(['user_id', 'ner_clusters']) \
    .with_columns(pl.col('ner_clusters').str.strip_chars_start('\" ').str.to_lowercase()) \
    .join(user_id_mapping, on='user_id', how='left') \
    .join(ner_mapping, left_on='ner_clusters', right_on='ner') \
    .select(['user_index', 'ner_index', 'user_id', 'article_ids_inview']) \
    .unique(['user_index', 'ner_index']) \
    .sort(['user_index', 'ner_index']).group_by('user_index').map_groups(
        lambda user_df: user_df.pipe(get_inview_scores, recommender=pure_svd, f_name='pure_svd_scores') \
            .pipe(get_inview_scores, recommender=item_knn, f_name='item_knn_scores')
    )

behaviors_ner_scores = behaviors_val.select(['impression_id', 'user_id', 'article_ids_inview']).explode('article_ids_inview') \
    .join(ner_inview_scores, on=['user_id', 'article_ids_inview'], how='left') \
    .with_columns(pl.col(['pure_svd_scores', 'item_knn_scores']).fill_null(0)) \
    .group_by(['impression_id', 'article_ids_inview']).agg(
        pl.col('pure_svd_scores').sum().alias('total_ner_svd_scores'),
        pl.col('pure_svd_scores').mean().alias('mean_ner_svd_scores'),
        pl.col('pure_svd_scores').max().alias('max_ner_svd_scores'),
        pl.col('item_knn_scores').sum().alias('total_ner_item_knn_scores'),
        pl.col('item_knn_scores').mean().alias('mean_ner_item_knn_scores'),
        pl.col('item_knn_scores').max().alias('max_ner_item_knn_scores'),
    ).with_columns(
        pl.col(['total_ner_svd_scores', 'mean_ner_svd_scores', 
                'total_ner_item_knn_scores', 'mean_ner_item_knn_scores',
                'max_ner_svd_scores', 'max_ner_item_knn_scores']).sum().over('impression_id').name.suffix('_max')
    ).with_columns(
        # l_inf normalization
        pl.col('total_ner_svd_scores').truediv(pl.col('total_ner_svd_scores_max')),
        pl.col('mean_ner_svd_scores').truediv(pl.col('mean_ner_svd_scores_max')),
        pl.col('max_ner_svd_scores').truediv(pl.col('max_ner_svd_scores_max')),
        pl.col('total_ner_item_knn_scores').truediv(pl.col('total_ner_item_knn_scores_max')),
        pl.col('mean_ner_item_knn_scores').truediv(pl.col('mean_ner_item_knn_scores_max')),
        pl.col('max_ner_item_knn_scores').truediv(pl.col('max_ner_item_knn_scores_max')),
    ).drop(['total_ner_svd_scores_max', 'mean_ner_svd_scores_max', 
            'max_ner_svd_scores_max', 'total_ner_item_knn_scores_max',
            'mean_ner_item_knn_scores_max', 'max_ner_item_knn_scores_max']) \
    .rename({'article_ids_inview': 'article'})
behaviors_ner_scores.head(5)

CPU times: user 3min 59s, sys: 2min 44s, total: 6min 43s
Wall time: 1min 50s


impression_id,article,total_ner_svd_scores,mean_ner_svd_scores,max_ner_svd_scores,total_ner_item_knn_scores,mean_ner_item_knn_scores,max_ner_item_knn_scores
u32,i32,f32,f32,f32,f32,f32,f32
101665143,9673564,-0.000208,-0.000625,0.0,0.0,0.0,0.0
495156171,9782745,0.422554,0.427991,0.407671,0.465658,0.455441,0.403262
88010264,9783657,-0.00293,-0.002955,0.002423,0.015528,0.013526,0.026389
130820222,9771113,0.139194,0.123333,0.071868,0.122218,0.10496,0.060637
18231388,9784281,0.127263,0.066722,0.118927,0.14862,0.084795,0.124174


In [51]:
%%time

STRONG_THR = 0.8

history_val = pl.concat(
    rows.with_columns(pl.col('article_id_fixed').list.len().alias('NumArticlesHistory')) \
        .explode(['article_id_fixed', 'impression_time_fixed', 'read_time_fixed', 'scroll_percentage_fixed']) \
        .sort(by=['user_id', 'impression_time_fixed']) \
        .with_columns(
            pl.col('impression_time_fixed').dt.weekday().alias('weekday'),
            pl.col('impression_time_fixed').dt.hour().alias('hour'),
        ).join(articles.select(['article_id', 'category', 'is_article_default', 'sentiment_label_int', 
                                'sentiment_score', 'entity_groups', 'topics']), 
               left_on='article_id_fixed', right_on='article_id', how='left') \
        .with_columns(
            (pl.col('sentiment_label_int') == 0).alias('is_neutral'),
            (pl.col('sentiment_label_int') == 1).alias('is_positive'),
            (pl.col('sentiment_label_int') == -1).alias('is_negative'),
            ((pl.col('sentiment_label_int') == 0) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_neutral'),
            ((pl.col('sentiment_label_int') == 1) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_positive'),
            ((pl.col('sentiment_label_int') == -1) & (pl.col('sentiment_score') > STRONG_THR)).alias('strong_negative'),
            pl.col('entity_groups').list.unique(),
        ).group_by('user_id').agg(
            pl.col('article_id_fixed'),
            pl.col('impression_time_fixed'),
            pl.col('category'),
            pl.col('NumArticlesHistory').first(),
            pl.col('read_time_fixed').median().alias('MedianReadTime'),
            pl.col('read_time_fixed').max().alias('MaxReadTime'),
            pl.col('read_time_fixed').sum().alias('TotalReadTime'),
            pl.col('scroll_percentage_fixed').median().alias('MedianScrollPercentage'),
            pl.col('scroll_percentage_fixed').max().alias('MaxScrollPercentage'),
            (pl.col('is_neutral').sum() / pl.col('NumArticlesHistory').first()).alias('NeutralPct'),
            (pl.col('is_positive').sum() / pl.col('NumArticlesHistory').first()).alias('PositivePct'),
            (pl.col('is_negative').sum() / pl.col('NumArticlesHistory').first()).alias('NegativePct'),
            (pl.col('strong_neutral').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongNeutral'),
            (pl.col('strong_positive').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongPositive'),
            (pl.col('strong_negative').sum() / pl.col('NumArticlesHistory').first()).alias('PctStrongNegative'),
            (1 - (pl.col('is_article_default').sum() / pl.col('NumArticlesHistory').first())).alias('PctNotDefaultArticles'),
            pl.col('category').mode().alias('MostFrequentCategory'),
            pl.col('weekday').mode().alias('MostFrequentWeekday'),
            pl.col('hour').mode().alias('MostFrequentHour'),
            pl.col('entity_groups').flatten(),
            pl.col('topics').flatten().alias('topics_flatten')
        ).pipe(add_topics_tf_idf_columns, topics_col='topics_flatten', vectorizer=vectorizer) \
        .drop('topics_flatten').with_columns(
            pl.col('MostFrequentCategory').list.first(),
            pl.col('MostFrequentWeekday').list.first(),
            pl.col('MostFrequentHour').list.first(),
        ).with_columns(
            (pl.col('entity_groups').list.count_matches(entity) / pl.col('NumArticlesHistory')).alias(f'{entity}Pct')
            for entity in unique_entities
        ).drop('entity_groups')
    for rows in tqdm.tqdm(history_val.iter_slices(1000), total=history_val.shape[0] // 1000)
)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
history_val = reduce_polars_df_memory_size(history_val)
history_val.head(2)

16it [00:14,  1.07it/s]                        



RAM memory GB usage = 9.748
Memory usage of dataframe is 36.51 MB
Memory usage after optimization is: 35.72 MB
Decreased by 2.2%
CPU times: user 17.9 s, sys: 1.24 s, total: 19.1 s
Wall time: 15.7 s


user_id,article_id_fixed,impression_time_fixed,category,NumArticlesHistory,MedianReadTime,MaxReadTime,TotalReadTime,MedianScrollPercentage,MaxScrollPercentage,NeutralPct,PositivePct,NegativePct,PctStrongNeutral,PctStrongPositive,PctStrongNegative,PctNotDefaultArticles,MostFrequentCategory,MostFrequentWeekday,MostFrequentHour,topics_flatten_tf_idf,PRODPct,ORGPct,LOCPct,MISCPct,EVENTPct,PERPct
u32,list[i32],list[datetime[μs]],list[i16],u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i16,i8,i8,list[f32],f32,f32,f32,f32,f32,f32
10623,"[9750749, 9685790, … 9779517]","[2023-05-05 07:51:01, 2023-05-05 07:51:36, … 2023-05-24 17:10:11]","[118, 457, … 414]",150,3.5,1563.0,9330.0,37.0,100.0,0.28,0.093333,0.626667,0.126667,0.06,0.546667,0.06,118,1,20,"[0.0, 0.042172, … 0.089681]",0.213333,0.913333,0.846667,0.266667,0.3,0.893333
12718,"[9749036, 9749025, … 9778718]","[2023-05-04 07:49:49, 2023-05-04 07:50:15, … 2023-05-24 07:07:25]","[118, 118, … 140]",114,7.0,748.0,3440.0,36.0,100.0,0.245614,0.052632,0.701754,0.087719,0.035088,0.614035,0.035088,118,4,7,"[0.0, 0.048524, … 0.217846]",0.254386,0.947368,0.815789,0.192982,0.114035,0.807018


In [52]:
last_history_df = history_val.with_columns(
    pl.col('impression_time_fixed').list.max().alias('last_history_impression_time'),
    pl.col('article_id_fixed').list.tail(1).alias('last_history_article'),
).select(['user_id', 'last_history_impression_time', 'last_history_article'])

last_session_time_df = behaviors_val.select(['session_id', 'user_id', 'impression_time', 'article_ids_inview', 'article_ids_clicked']) \
    .group_by('session_id').agg(
        pl.col('user_id').first(), 
        pl.col('impression_time').max().alias('session_time'), 
        pl.col('article_ids_inview').flatten().alias('all_seen_articles'),
        (pl.col('impression_time').max() - pl.col('impression_time').min()).dt.total_minutes().alias('session_duration'),
    ).with_columns(
        pl.col(['session_time', 'session_duration']).shift(1).over('user_id').name.prefix('last_'),
        pl.col('all_seen_articles').list.unique().shift(1).over('user_id'),
        pl.col('session_duration').rolling_mean(100, min_periods=1).over('user_id').alias('mean_prev_sessions_duration'),
    ).with_columns(pl.col(['last_session_duration']).fill_null(0)) \
    .join(last_history_df, on='user_id', how='left') \
    .with_columns(
        pl.col('last_session_time').fill_null(pl.col('last_history_impression_time')),
        pl.col('all_seen_articles').fill_null(pl.col('last_history_article')),
    ).select(['session_id', 'last_session_time', 'last_session_duration',
              'all_seen_articles', 'mean_prev_sessions_duration'])

In [53]:
%%time

NPRATIO = 2

users_mean_trendiness_scores, topics_mean_trendiness_scores = _preprocessing_history_trendiness_scores(
        history=history_val, articles=articles)
topic_mean_delays, user_mean_delays = _preprocessing_mean_delay_features(
    articles=articles, history=history_val)
windows, user_windows, user_topics_windows, user_category_windows = _preprocessing_window_features(
    history=history_val, articles=articles)
articles_endorsement = _preprocessing_article_endorsement_feature(
    behaviors=behaviors_val, period="10h")
normalized_endorsement = normalize_endorsement(articles_endorsement)
articles_endorsement_articleuser = _preprocessing_article_endorsement_feature_by_article_and_user(
    behaviors=behaviors_val, period="20h")
normalized_endorsement_articleuser = normalize_endorsement_by_article_and_user(articles_endorsement_articleuser)

history_counts=history_val.select(['article_id_fixed']).explode('article_id_fixed').group_by('article_id_fixed').len() \
.rename({'article_id_fixed':'article','len':'clicked_count'})

behaviors_counts=behaviors_val.select('article_ids_inview').explode('article_ids_inview').group_by('article_ids_inview').len() \
.rename({'article_ids_inview':'article','len':'inview_count'})

# dropping duplicates that can occur because of sampling strategy
val_ds = behaviors_val.pipe(create_binary_labels_column, shuffle=True, seed=123) \
    .with_columns(pl.col('gender').fill_null(2)) \
    .with_columns(pl.col('postcode').fill_null(5)) \
    .select(['impression_id', 'article_ids_inview', 'impression_time', 'labels', 
             'device_type', 'read_time', 'scroll_percentage', 'user_id', 'is_sso_user', 'gender',
             'age', 'is_subscriber', 'session_id', 'postcode']) \
    .explode(['article_ids_inview', 'labels']) \
    .rename({'article_ids_inview': 'article', 'labels': 'target'}) \
    .with_columns(pl.col('article').cast(pl.Int32)) \
    .pipe(add_trendiness_feature, articles=articles, period='1d') \
    .rename({'trendiness_score': 'trendiness_score_1d'}) \
    .pipe(add_trendiness_feature, articles=articles, period='3d') \
    .rename({'trendiness_score': 'trendiness_score_3d'}) \
    .pipe(add_trendiness_feature, articles=articles, period='5d') \
    .rename({'trendiness_score': 'trendiness_score_5d'}) \
    .pipe(add_trendiness_feature_leak, articles=articles, period='3d') \
    .rename({'trendiness_score_leak':'trendiness_score_3d_leak'}) \
    .unique(['impression_id', 'article']) \
    .with_columns(
        pl.col('impression_time').dt.weekday().alias('weekday'),
        pl.col('impression_time').dt.hour().alias('hour'),
        pl.col('article').cast(pl.Int32),
        (
            pl.col('trendiness_score_1d') / 
            pl.col('trendiness_score_3d')
        ).alias('trendiness_score_1d/3d'),
        (
            pl.col('trendiness_score_1d') / 
            pl.col('trendiness_score_5d')
        ).alias('trendiness_score_1d/5d'),
        (
            pl.col('trendiness_score_3d') / 
            pl.col('trendiness_score_3d').max().over(pl.col('impression_time').dt.date())
        ).alias('normalized_trendiness_score_overall'),
    ).join(articles.select(['article_id', 'premium', 'published_time', 'category',
                            'sentiment_score', 'sentiment_label', 'entity_groups',
                            'num_images', 'title_len', 'subtitle_len', 'body_len','num_topics',
                            'total_pageviews', 'total_inviews', 'total_read_time', 
                            'total_pageviews/inviews', 'article_type']),
           left_on='article', right_on='article_id', how='left') \
    .with_columns(
        (pl.col('impression_time') - pl.col('published_time')).dt.total_days().alias('article_delay_days'),
        (pl.col('impression_time') - pl.col('published_time')).dt.total_hours().alias('article_delay_hours')
    ).with_columns(
        pl.col('entity_groups').list.contains(entity).alias(f'Entity_{entity}_Present')
        for entity in unique_entities
    ).drop('entity_groups') \
    .join(behaviors_ner_scores, on=['impression_id', 'article'], how='left') \
    .pipe(add_history_trendiness_scores_feature, articles=articles, 
          users_mean_trendiness_scores=users_mean_trendiness_scores,
          topics_mean_trendiness_scores=topics_mean_trendiness_scores, topics=topics) \
    .pipe(add_mean_delays_features, articles=articles,
          topic_mean_delays=topic_mean_delays, user_mean_delays=user_mean_delays) \
    .pipe(add_window_features, articles=articles, user_windows=user_windows,
          user_category_windows=user_category_windows, 
          user_topics_windows=user_topics_windows, windows=windows) \
    .pipe(add_trendiness_feature_categories, articles=articles) \
    .pipe(add_article_endorsement_feature, normalized_endorsement=normalized_endorsement) \
    .pipe(add_article_endorsement_feature_by_article_and_user, normalized_endorsement = normalized_endorsement_articleuser) \
    .pipe(add_article_endorsement_feature_leak, normalized_endorsement = normalized_endorsement) \
    .rename({
        "endorsement_10h_diff_rolling_right":"endorsement_10h_leak_diff_rolling",
        "endorsement_macd_right":"endorsement_leak_macd",
        "endorsement_quantile_norm_10h_right":"endorsement_quantile_norm_10h_leak",
        "normalized_endorsement_10h_rolling_max_ratio_right":"normalized_endorsement_10h_leak_rolling_max_ratio"
    }).join(history_counts, on='article',how='left') \
    .join(behaviors_counts , on='article', how='left') \
    .join(last_session_time_df, on='session_id', how='left').with_columns(
        (pl.col('impression_time') - pl.col('last_session_time')).dt.total_hours().alias('last_session_time_hour_diff'),
        ((pl.col('last_session_time') - pl.col('published_time')).dt.total_hours() > 0).alias('is_new_article'),
        pl.col('all_seen_articles').list.contains(pl.col('article')).alias('is_already_seen_article'),
    ).drop(['published_time', 'session_id', 'all_seen_articles', 'last_session_time']) \
    .join(published_category_popularity, how='left', right_on=['published_date', 'category'],
          left_on=[pl.col('impression_time').dt.date() - pl.duration(days=1), 'category']) \
    .rename({'category_daily_pct': 'yesterday_category_daily_pct'}) \
    .with_columns(pl.col('yesterday_category_daily_pct').fill_null(0))

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)

val_ds = reduce_polars_df_memory_size(val_ds)
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
val_ds.head()

16it [00:23,  1.44s/it]                        
2205it [00:04, 537.38it/s]                          
16it [00:01, 10.67it/s]                        
16it [00:01,  8.50it/s]                        
100%|██████████| 507/507 [00:04<00:00, 102.96it/s]



RAM memory GB usage = 11.75
Memory usage of dataframe is 1314.28 MB
Memory usage after optimization is: 735.73 MB
Decreased by 44.0%

RAM memory GB usage = 11.19
CPU times: user 13min 2s, sys: 1min 7s, total: 14min 10s
Wall time: 4min 15s


impression_id,article,user_id,impression_time,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,…,window_0_history_length,window_1_history_length,window_2_history_length,window_3_history_length,is_inside_window_0,is_inside_window_1,is_inside_window_2,is_inside_window_3,window_category_score,window_topics_score,trendiness_score_category,endorsement_10h,normalized_endorsement_10h,endorsement_10h_diff_rolling,endorsement_macd,endorsement_quantile_norm_10h,normalized_endorsement_10h_rolling_max_ratio,endorsement_20h_articleuser,normalized_endorsement_articleuser_20h,endorsement_20h_articleuser_diff_rolling,endorsement_articleuser_macd,endorsement_articleuser_quantile_norm_20h,normalized_endorsement_articleuser_20h_rolling_max_ratio,endorsement_10h_leak,normalized_endorsement_10h_right,endorsement_10h_leak_diff_rolling,endorsement_leak_macd,endorsement_quantile_norm_10h_leak,normalized_endorsement_10h_leak_rolling_max_ratio,clicked_count,inview_count,last_session_duration,mean_prev_sessions_duration,last_session_time_hour_diff,is_new_article,is_already_seen_article,yesterday_category_daily_pct
u32,i32,u32,date,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,…,i16,i16,i16,i16,i8,i8,i8,i8,i16,i16,i8,u16,f32,f32,f32,f32,f32,u8,f32,f32,f32,f32,f32,i16,f32,f32,f32,f32,f32,u32,u16,i8,f32,i16,bool,bool,f32
497194197,9777910,393380,2023-05-24,0,2,30.0,null,false,2,null,false,5,4,9,17,8,4,7,0.444444,0.235294,0.016484,false,457,0.9306,"""Neutral""",1,8,18,92,2,93958,440140,7.265285e6,0.213473,"""article_default""",0,…,476,225,62,21,1,0,0,0,2,6,9,2,0.012821,0.0,0.0,0.666667,0.333333,1,0.05,0.0,0.0,1.0,1.0,264,0.003372,4.5,2.5,0.15493,0.115284,1331,367,3,2.84,-74,true,false,0.021127
565885711,9780193,1389619,2023-05-24,1,1,27.0,null,true,0,null,false,5,1,5,9,3,4,7,0.2,0.111111,0.009158,false,118,0.5032,"""Neutral""",0,7,13,225,1,58440,368985,3.245847e6,0.15838,"""article_default""",0,…,393,281,104,241,1,0,0,0,205,12,109,4,0.025641,0.0,0.0,1.333333,1.0,1,0.058824,0.0,0.0,1.0,1.0,1048,0.009809,4.5,2.5,0.729805,0.477666,803,1112,0,0.833333,-41,true,false,0.239437
497194197,9777769,393380,2023-05-24,0,2,30.0,null,false,2,null,false,5,88,243,378,205,4,7,0.36214,0.232804,0.445055,false,118,0.8781,"""Neutral""",3,8,14,333,3,116897,509990,5.323702e6,0.229214,"""article_default""",0,…,476,225,62,21,1,0,0,0,133,294,109,1,0.00641,0.0,0.0,0.333333,0.166667,1,0.05,0.0,0.0,1.0,1.0,86,0.000863,4.5,2.5,0.046943,0.031769,1808,126,3,2.84,-74,true,false,0.239437
567281774,9777233,589220,2023-05-24,0,2,19.0,null,false,2,null,false,5,94,275,460,244,4,7,0.341818,0.204348,0.503663,false,142,0.8275,"""Positive""",1,4,21,249,3,42789,280339,2.593098e6,0.152633,"""article_default""",1,…,125,215,74,87,1,0,0,0,20,69,112,1,0.00641,0.0,0.0,0.333333,0.25,1,0.166667,0.0,0.0,1.0,1.0,20,0.000225,4.5,2.5,0.010433,0.008382,601,133,0,1.176471,-47,true,false,0.239437
497194197,9779891,393380,2023-05-24,0,2,30.0,null,false,2,null,false,5,158,490,835,451,4,7,0.322449,0.189222,0.897436,false,142,0.9534,"""Positive""",1,8,17,206,5,18946,216460,927517.0,0.087527,"""article_default""",0,…,476,225,62,21,1,0,0,0,105,472,112,2,0.012821,0.0,0.0,0.666667,0.333333,1,0.05,0.0,0.0,1.0,1.0,127,0.001955,5.2,2.8,0.068206,0.044908,204,263,3,2.84,-74,true,false,0.239437


In [54]:
val_ds.shape

(2928942, 93)

In [55]:
prev_columns = [c for c in val_ds.columns if c not in ['impression_id', 'article', 'impression_time']]

val_ds = pl.concat(
    rows.join(history_val.select(['user_id', 'article_id_fixed', 'impression_time_fixed']), on='user_id', how='left') \
        .join(articles.select(['article_id', 'topics', 'entity_groups', 'topics_idf'] + topic_model_columns), 
              left_on='article', right_on='article_id', how='left') \
        .explode(['article_id_fixed', 'impression_time_fixed']) \
        .join(articles.select(['article_id', 'topics', 'entity_groups'] + topic_model_columns), 
              left_on='article_id_fixed', right_on='article_id', how='left') \
        .rename({'topics_right': 'topics_history', 'entity_groups_right': 'entity_groups_history'}) \
        .rename({f'topic_model_embedding_{i}_right': f'topic_model_embedding_{i}_history' for i in range(N_COMPONENTS)}) \
        .with_columns(
            # 1/delay gives the weight for the weighted mean of the lda embeddings
            (1 / (pl.col('impression_time') - pl.col('impression_time_fixed')).dt.total_hours().sqrt()).alias('history_weight'),
            (pl.col("topics").list.set_intersection(pl.col("topics_history")).list.len().truediv(
                pl.col("topics").list.set_union(pl.col("topics_history")).list.len()
            )).alias("JS"),
            pl.col('entity_groups').list.set_intersection(pl.col('entity_groups_history')).list.len().alias('common_entities'),
        ).drop(['entity_groups_history', 'entity_groups', 'topics', 'topics_history']) \
        .with_columns(
            # summing delays to normalize them before weighted mean
            pl.col('history_weight').sum().over(['impression_id', 'article']).alias('history_weight_sum'),
            *[pl.col(x).mul(pl.col(f'{x}_history')).alias(f'{x}_dot') for x in topic_model_columns],
            *[pl.col(x).mul(pl.col(x)) for x in topic_model_columns],
            *[pl.col(f'{x}_history').mul(pl.col(f'{x}_history')).alias(f'{x}_history_square') for x in topic_model_columns],
        ).with_columns(
            # weights now sum to 1
            pl.col('history_weight').truediv(pl.col('history_weight_sum')),
            pl.sum_horizontal(topic_model_columns).sqrt().alias('topic_model_norm'),
            pl.sum_horizontal([f'{x}_history_square' for x in topic_model_columns]).sqrt().alias('topic_model_history_norm'),
            pl.sum_horizontal([f'{x}_dot' for x in topic_model_columns]).alias('topic_model_dot'),
        ).with_columns(
            *[pl.col(f'{x}_history').mul(pl.col('history_weight')).alias(f'{x}_history_weighted') for i, x in enumerate(topic_model_columns)],
            pl.col('topic_model_dot').truediv(pl.col('topic_model_norm').mul('topic_model_history_norm')).alias('topic_model_cosine')
        ).group_by(['impression_id', 'article']).agg(
            pl.col(prev_train_columns).first(),
            pl.col('topics_idf').first(),
            pl.col('common_entities').mean().alias('MeanCommonEntities'),
            pl.col('common_entities').max().alias('MaxCommonEntities'),
            pl.col("JS").mean().alias("mean_JS"),
            pl.col("JS").min().alias("min_JS"),
            pl.col("JS").max().alias("max_JS"),
            pl.col("JS").std().alias("std_JS"),
            pl.col("JS").mul(pl.col("history_weight")).sum().alias("weighted_mean_JS"),
            pl.col("topic_model_cosine").mean().alias("mean_topic_model_cosine"),
            pl.col("topic_model_cosine").min().alias("min_topic_model_cosine"),
            pl.col("topic_model_cosine").max().alias("max_topic_model_cosine"),
            pl.col("topic_model_cosine").std().alias("std_topic_model_cosine"),
            pl.col("topic_model_cosine").mul(pl.col("history_weight")).sum().alias("weighted_mean_topic_model_cosine"),
            *[pl.col(f'{x}_history').mean().alias(f'lda_{i}_history_mean') for i, x in enumerate(topic_model_columns)],
            *[pl.col(f'{x}_history_weighted').sum().alias(f'lda_{i}_history_weighted_mean') for i, x in enumerate(topic_model_columns)]
        ).join(history_val.drop(['article_id_fixed', 'impression_time_fixed']), on='user_id', how='left') \
        .with_columns(
            pl.struct(['topics_idf', 'topics_flatten_tf_idf']).map_elements(
                lambda x: cosine_similarity(x['topics_idf'], x['topics_flatten_tf_idf']), return_dtype=pl.Float64
            ).cast(pl.Float32).alias('topics_cosine'),
            (pl.col('category') == pl.col('MostFrequentCategory')).alias('IsFavouriteCategory'),
            pl.col('category_right').list.n_unique().alias('NumberDifferentCategories'),
            list_pct_matches_with_col('category_right', 'category').alias('PctCategoryMatches'),
        ).with_columns(
            [list_pct_matches_with_constant('category_right', c).alias(f'Category_{c_str}_Pct')
             for c, c_str in unique_categories.items()]
        ).drop(['topics_idf', 'topics_flatten', 'topics_flatten_tf_idf', 'category_right'])
    for rows in tqdm.tqdm(val_ds.iter_slices(10000), total=val_ds.shape[0] // 10000)
)

val_ds = reduce_polars_df_memory_size(val_ds)
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
val_ds.head()

293it [34:42,  7.11s/it]


Memory usage of dataframe is 1925.91 MB
Memory usage after optimization is: 1492.96 MB
Decreased by 22.5%

RAM memory GB usage = 13.36


impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,MostFrequentWeekday,MostFrequentHour,PRODPct,ORGPct,LOCPct,MISCPct,EVENTPct,PERPct,topics_cosine,IsFavouriteCategory,NumberDifferentCategories,PctCategoryMatches,Category_underholdning_Pct,Category_ferie_Pct,Category_bibliotek_Pct,Category_penge_Pct,Category_side9_Pct,Category_krimi_Pct,Category_forbrug_Pct,Category_auto_Pct,Category_dagsorden_Pct,Category_musik_Pct,Category_haandvaerkeren_Pct,Category_podcast_Pct,Category_plus_Pct,Category_horoskoper_Pct,Category_vin_Pct,Category_biler_Pct,Category_incoming_Pct,Category_services_Pct,Category_nyheder_Pct,Category_sport_Pct,Category_om_ekstra_bladet_Pct,Category_opinionen_Pct,Category_nationen_Pct,Category_video_Pct,Category_sex_og_samliv_Pct
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,i8,i8,f32,f32,f32,f32,f32,f32,f32,bool,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
116609521,9777026,1515189,0,2,10.0,null,false,2,null,false,5,125,367,589,336,4,7,0.340599,0.212224,0.672161,false,414,0.5496,"""Neutral""",2,9,16,225,6,197066,748840,1.0179357e7,0.263162,"""article_default""",1,45,…,7,21,0.314286,0.921429,0.782143,0.242857,0.260714,0.896429,0.420377,false,9,0.971429,0.185714,0.0,0.0,0.021429,0.0,0.128571,0.042857,0.0,0.0,0.039286,0.0,0.0,0.0,0.0,0.0,0.003571,0.0,0.0,0.214286,0.339286,0.0,0.0,0.025,0.0,0.0
487191524,9779737,1978435,0,1,9.0,null,true,2,null,true,5,71,201,296,160,4,7,0.353234,0.239865,0.368132,false,140,0.9939,"""Negative""",0,6,19,127,5,195734,618596,9.766141e6,0.316417,"""article_default""",0,12,…,5,6,0.304493,0.898502,0.793677,0.231281,0.206323,0.873544,0.383397,false,12,0.981697,0.159734,0.0,0.0,0.03827,0.003328,0.199667,0.019967,0.0,0.0,0.023295,0.0,0.0,0.0,0.0,0.0,0.001664,0.0,0.0,0.349418,0.171381,0.0,0.001664,0.02995,0.0,0.001664
220893386,9778939,2288664,0,2,57.0,null,false,2,null,false,5,110,319,484,255,4,7,0.344828,0.227273,0.584249,true,414,0.9926,"""Negative""",3,8,18,776,5,144704,1279706,8.283967e6,0.113076,"""article_default""",0,15,…,1,7,0.217822,0.881188,0.762376,0.267327,0.267327,0.851485,0.254449,false,8,0.930693,0.09901,0.0,0.0,0.059406,0.0,0.089109,0.0,0.0,0.0,0.019802,0.0,0.0,0.0,0.0,0.0,0.009901,0.0,0.0,0.366337,0.336634,0.0,0.0,0.019802,0.0,0.0
84218843,7310404,619911,0,1,52.0,null,false,2,null,false,5,19,85,123,51,4,7,0.223529,0.154472,0.155678,true,457,0.6599,"""Neutral""",11,10,33,3021,3,null,null,null,null,"""article_default""",1701,40845,…,3,8,0.5,1.0,0.75,0.25,0.3,0.7,0.033559,false,5,0.75,0.0,0.0,0.0,0.45,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.25,0.0,0.0,0.05,0.0,0.0
497195044,9547869,396237,0,1,13.0,null,false,2,null,false,5,22,81,116,43,4,7,0.271605,0.189655,0.148352,true,457,0.9015,"""Positive""",3,5,32,811,3,null,null,null,null,"""article_default""",143,3433,…,1,8,0.232323,0.888889,0.742424,0.199495,0.209596,0.833333,0.081745,false,9,0.979798,0.116162,0.0,0.0,0.025253,0.0,0.184343,0.015152,0.0,0.0,0.032828,0.0,0.0,0.0,0.0,0.0,0.002525,0.0,0.0,0.265152,0.295455,0.0,0.0,0.063131,0.0,0.0


In [56]:
for expressions_group in tqdm.tqdm(expressions):
    val_ds = val_ds.with_columns(expressions_group)
    val_ds = reduce_polars_df_memory_size(val_ds)

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
val_ds.head(2)

  0%|          | 0/12 [00:00<?, ?it/s]

Memory usage of dataframe is 1807.55 MB


  8%|▊         | 1/12 [00:06<01:12,  6.61s/it]

Memory usage after optimization is: 1695.82 MB
Decreased by 6.2%
Memory usage of dataframe is 1729.69 MB


 17%|█▋        | 2/12 [00:10<00:48,  4.85s/it]

...

 92%|█████████▏| 11/12 [01:12<00:04,  4.62s/it]

Memory usage after optimization is: 2864.89 MB
Decreased by 0.9%
Memory usage of dataframe is 2887.23 MB


100%|██████████| 12/12 [01:15<00:00,  6.26s/it]

Memory usage after optimization is: 2876.06 MB
Decreased by 0.4%

RAM memory GB usage = 13.69


impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,trendiness_score_3d_leak,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,…,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_total_read_time,entropy_impression_endorsement_10h_leak,entropy_impression_trendiness_score_3d_leak,entropy_impression_clicked_count,entropy_impression_inview_count,trendiness_score_3d_minus_median_impression,trendiness_score_5d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,total_read_time_minus_median_impression,endorsement_10h_leak_minus_median_impression,trendiness_score_3d_leak_minus_median_impression,clicked_count_minus_median_impression,inview_count_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,endorsement_20h_articleuser_l_inf_article_user
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u8,u8,f32
116609521,9777026,1515189,0,2,10.0,null,false,2,null,false,5,125,367,589,336,4,7,0.340599,0.212224,0.672161,false,414,0.5496,"""Neutral""",2,9,16,225,6,197066,748840,1.0179357e7,0.263162,"""article_default""",1,45,…,2.246055,2.372406,2.139062,2.274145,2.289888,2.090699,2.182534,124.0,211.0,-40.0,0.154064,0.045605,0.055349,0.156275,35.0,160231.0,344576.0,-51.0,0.050976,7.435077e6,-169.0,131.0,1897.0,1623.0,0.587503,0.555269,0.894016,0.582222,0.255682,0.456204,0.472471,0.853686,0.497013,5,2,1,0.5
487191524,9779737,1978435,0,1,9.0,null,true,2,null,true,5,71,201,296,160,4,7,0.353234,0.239865,0.368132,false,140,0.9939,"""Negative""",0,6,19,127,5,195734,618596,9.766141e6,0.316417,"""article_default""",0,12,…,2.398186,2.576786,2.112226,2.327237,2.421293,2.113113,2.188842,-35.0,-63.5,-12.0,0.09931,0.008044,-0.044526,0.087374,0.0,64057.0,89228.0,-1.0,0.013475,1.250428e6,3.5,-20.0,2110.5,-127.0,0.740284,0.600033,0.813735,0.700583,0.071856,0.241643,0.43552,0.826185,0.446626,7,3,2,1.0


In [57]:
del history_val
del articles
del behaviors_val

del behaviors_ner_scores
del published_category_popularity, last_session_time_df
del users_mean_trendiness_scores, topics_mean_trendiness_scores
del topic_mean_delays, user_mean_delays
del windows, user_windows, user_topics_windows, user_category_windows
del articles_endorsement
del normalized_endorsement
del articles_endorsement_articleuser
del normalized_endorsement_articleuser
del history_counts, behaviors_counts

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 13.43


In [58]:
val_ds.shape

(2928942, 298)

In [59]:
X_val = val_ds.drop(['target', 'impression_id', 'article', 'user_id']).to_pandas()
X_val[numerical_columns] = X_val[numerical_columns].replace([-np.inf, np.inf], np.nan)
for col in numerical_columns:
    X_val[col] = X_val[col].fillna(min_values[col]).astype(np.float32)
X_val_numerical = scaler.transform(X_val[numerical_columns].values).astype(np.float32)
X_val_categorical = categorical_encoder.transform(X_val[categorical_columns]).astype(np.float32)

val_ds = val_ds.select(['impression_id', 'target', 'user_id', 'article'])
gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)

X_val = np.concatenate([X_val_numerical, X_val_categorical], axis=1)
y_val = val_ds['target'].to_numpy()

del X_numerical, X_val_numerical, X_val_categorical
gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)

val_ds = val_ds.with_columns(pl.Series(model.predict_proba(X_val)[:, 1]).alias('prediction'))
val_ds.select(['impression_id', 'target', 'prediction'])

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [14, 17] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



RAM memory GB usage = 20.41

RAM memory GB usage = 15.92


impression_id,target,prediction
u32,i8,f64
116609521,0,0.36727
487191524,0,0.360688
220893386,0,0.374432
84218843,0,0.260724
497195044,0,0.259963
…,…,…
316579785,0,0.412882
486286396,0,0.265398
222961173,0,0.4925


In [60]:
del X_val

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 11.85


In [61]:
evaluation_ds = val_ds.group_by('impression_id').agg(pl.col('target'), pl.col('prediction'))
evaluation_ds

impression_id,target,prediction
u32,list[i8],list[f64]
322704021,"[0, 0, … 0]","[0.320634, 0.391954, … 0.319963]"
152988256,"[1, 0, … 0]","[0.42891, 0.286791, … 0.283859]"
200888842,"[0, 0, … 0]","[0.222776, 0.302507, … 0.312552]"
391562232,"[0, 1, … 0]","[0.595555, 0.28956, … 0.478465]"
152366003,"[0, 0, … 1]","[0.395961, 0.262594, … 0.5044]"
…,…,…
417170090,"[0, 0, … 0]","[0.588724, 0.443262, … 0.404806]"
577026326,"[0, 0, … 0]","[0.41902, 0.257503, … 0.239562]"
94309163,"[1, 0, … 0]","[0.444968, 0.427745, … 0.382964]"


In [62]:
%%time

met_eval = MetricEvaluator(
    labels=evaluation_ds['target'].to_list(),
    predictions=evaluation_ds['prediction'].to_list(),
    metric_functions=[
        AucScore(),
        MrrScore(),
        NdcgScore(k=5),
        NdcgScore(k=10),
    ],
)
met_eval.evaluate()

CPU times: user 7min 26s, sys: 546 ms, total: 7min 27s
Wall time: 7min 27s


<MetricEvaluator class>: 
 {
    "auc": 0.7938262907993072,
    "mrr": 0.580262284603487,
    "ndcg@5": 0.6455744582178581,
    "ndcg@10": 0.6726477626392926
}

In [63]:
import joblib

joblib.dump(model, '/kaggle/working/fast_rgf.joblib')
joblib.dump(categorical_encoder, '/kaggle/working/categorical_encoder.joblib')
joblib.dump(scaler, '/kaggle/working/scaler.joblib')
joblib.dump(min_values, '/kaggle/working/min_values_train.joblib')
val_ds.write_parquet('/kaggle/working/fast_rgf_predictions.parquet')